In [ ]:
# Remove this
#! pip install torch==2.3 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
! pip install --upgrade torch torchvision==0.15.0+cu118 --index-url https://download.pytorch.org/whl/cu118

#! pip install stacchip
#! pip install geopandas

! pip install contextily

In [4]:

import warnings

import geoarrow.pyarrow as ga
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pystac_client
import requests
import torch
import yaml
from box import Box

import stacchip.indexer
from stacchip.chipper import Chipper
from stacchip.indexer import Sentinel2Indexer
from torchvision.transforms import v2

import geopandas as gpd
import pandas as pd

from shapely.geometry import Polygon
import matplotlib.pyplot as plt

import math
import os
import random

import timm
import torch
import torch.nn.functional as F
from einops import rearrange, reduce, repeat
from torch import nn
from torchvision.transforms import v2

from src.backbone import Transformer
from src.factory import DynamicEmbedding
from src.utils import posemb_sincos_2d_with_gsd

torch.set_float32_matmul_precision("medium")
os.environ["TORCH_CUDNN_V8_API_DISABLED"] = "1"

#from src.model import ClayMAEModule

import contextily as cx
import rasterio
from rasterio.plot import show

from shapely.geometry import box

import sklearn
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from shapely import wkb

import stackstac
import pystac_client
import numpy as np
import dask
from rasterio.enums import Resampling
from rasterstats import zonal_stats
import xarray as xr
import rioxarray as rxr

import torch.nn as nn

import shapely
from shapely.geometry import Polygon
from rasterio.features import rasterize

warnings.filterwarnings("ignore")

### Load mining data and visualize

In [ ]:
# Load mine polygons from Tang dataset and remove Z coordinate
mine_poly = gpd.read_file('data/mine_poly_shp').to_crs(epsg=4326)

def convert_3D_2D(geometry):
    '''
    Takes a GeoSeries of 3D Multi/Polygons (has_z) and returns a list of 2D Multi/Polygons
    From : https://gist.github.com/rmania/8c88377a5c902dfbc134795a7af538d8
    '''
    new_geo = []
    for p in geometry:
        if p.has_z:
            if p.geom_type == 'Polygon':
                lines = [xy[:2] for xy in list(p.exterior.coords)]
                new_p = Polygon(lines)
                new_geo.append(new_p)
        else:
            print('no z')
    return new_geo

mine_poly.geometry = convert_3D_2D(mine_poly.geometry) 

print('Number of mine features:')
print(len(mine_poly))


In [ ]:
# Load mine point data
mine_pts = gpd.read_file('data/pitlakes').to_crs(epsg=4326)

# Print info about dataset
print(mine_pts['Material'].value_counts())
print(len(mine_pts))
print(mine_pts['ID'].value_counts())

In [5]:
# Load Aquarry QA'ed data
psql_bboxes = gpd.read_file('data/aquarry_psql_layer.csv')
psql_bboxes = gpd.GeoDataFrame(psql_bboxes,geometry = psql_bboxes['field_8'].apply(wkb.loads),crs='epsg:4326') # Convert WKB geometry

psql_bboxes.columns = ['cc', 'objectid', 'score', 'label', 'dataset', 'area', 'volume', 'wkb_geometry','field_9', 'category', 'field_11', 'geometry']

In [ ]:
# Subset US QA data
US_QA = psql_bboxes[psql_bboxes['cc'] == 'US']
print(US_QA['category'].value_counts())

f"Number of confirmed mine features: {len(US_QA['category']=='a')}"

# Write to shapefile to inspect
#US_QA.to_file('data/US_QA')

#### Assess distribution of pits

In [ ]:
QA_global_mines = psql_bboxes[psql_bboxes['category']=='a']

QA_global_mines_gdf = gpd.GeoDataFrame(QA_global_mines, geometry = 'geometry', crs = 4326)

QA_global_mines_gdf = QA_global_mines_gdf.to_crs(epsg=9822)

pit_areas = QA_global_mines_gdf.geometry.area

pit_areas.hist(bins=10)

print(len(pit_areas))

print(pit_areas.min())
print(pit_areas.max())
print(pit_areas.median())
print(pit_areas.mean())

print(math.sqrt(pit_areas.median()))

In [ ]:
pit_areas_majority = pit_areas[pit_areas<(0.008*1e8)]
print(len(pit_areas_majority)/len(pit_areas))

plt.hist(pit_areas_majority,bins=10)
plt.title('Histogram of pit lake areas')
#plt.xticks([0, 1, 2], ['Low', 'Medium', 'High'])

#plt.show()

mode_length = math.sqrt(0.1*1e6)
print(mode_length)
print(math.sqrt(800000))
print(len(pit_areas[pit_areas<(100000)]))

import scipy
print(np.percentile(pit_areas, 25))
print(np.percentile(pit_areas, 75))


In [ ]:
# Visualize with satellite basemap
providers = cx.providers.flatten()

# Visualize mine point data
ax = mine_pts.plot(column = 'Material', 
    markersize = 2,
    color = 'blue', 
    legend = True, 
    legend_kwds={'bbox_to_anchor': (1, 1)},
    figsize = (10,10),
    alpha = 0.5)
cx.add_basemap(ax, crs=mine_pts.crs, source=providers['NASAGIBS.BlueMarble'])

mine_poly.plot(ax=ax, color = 'red', linewidth = 6)

#### Filter QA'ed dataset

In [ ]:
states = gpd.read_file('data/state_boundaries')

# Get only QA'ed pits that are 0.5 or greater (i.e. have been QA'ed and/or higher likelihood)
print(f'Number of features in US dataset: {len(US_QA)}')
US_QA['score'] = US_QA['score'].apply(lambda score: float(score))
US_QA_filtered = US_QA[US_QA['score'] > 0.5]
US_QA_filtered['feature_idx'] = US_QA_filtered.reset_index(drop=True).index # Use for dissolve on spatial joining
print(f'Number of features in US with > 50% probability: {len(US_QA_filtered)}')


# Get only states with pit lakes
states['geometry'] = states['geometry'].apply(lambda geom: gpd.GeoSeries([geom]).unary_union)
states = gpd.GeoDataFrame(states, geometry='geometry', crs=states.crs).to_crs(epsg = 4326)
states = states[~states['STUSPS'].isin(['VI','PR','GU','MP','AS'])]

states_with_pits = gpd.sjoin(states, US_QA_filtered, how = 'left', predicate = 'intersects')
states_with_pits = states_with_pits[~states_with_pits['score'].isna()]

# Take out states with less than 5 pit lakes
states_with_enough_pits = states_with_pits['STUSPS'].value_counts() > 5
states_list = states_with_enough_pits[states_with_enough_pits].index.to_list()

states_with_low_pits = states_with_pits['STUSPS'].value_counts() < 5
print(f'States to remove: {states_with_low_pits[states_with_low_pits].index}')

states_filtered = states_with_pits[states_with_pits['STUSPS'].isin(states_list)].drop_duplicates('geometry')[['STUSPS','geometry']]

states_geometries = states[states['STUSPS'].isin(states_list)]
print(f'# of states being used: {len(states_geometries)}')

### State by state approach

#### Ohio, Kentucky, West Virginia

In [ ]:
# Get pit lakes in OH, KY, WV

states = gpd.read_file('data/state_boundaries')
oh = states[states['STUSPS']=='OH']
oh = oh.to_crs(epsg = 4326).geometry.unary_union

oh_qa = US_QA[US_QA.geometry.intersects(oh)]
print(f'OH QAed mines: {(oh_qa['category'] == 'a').sum()}')

ky = states[states['STUSPS']=='KY']
ky = ky.to_crs(epsg = 4326).geometry.unary_union

ky_qa = US_QA[US_QA.geometry.intersects(ky)]
print(f'KY QAed mines: {(ky_qa['category'] == 'a').sum()}')

wv = states[states['STUSPS']=='WV']
wv = wv.to_crs(epsg = 4326).geometry.unary_union
wv_qa = US_QA[US_QA.geometry.intersects(wv)]
print(f'WV QAed mines: {(wv_qa['category'] == 'a').sum()}')

#### Arizona

In [ ]:
# Get pit lakes in AZ

states = gpd.read_file('data/state_boundaries')
az = states[states['STUSPS']=='AZ']
az = az.to_crs(epsg = 4326).geometry.unary_union

#az_mines_pts = mine_pts[mine_pts.geometry.within(az)]
#az_mines_poly = mine_poly[mine_poly.geometry.intersects(az)]
#az_mines_intersect = mine_pts_polys[mine_pts_polys.geometry.intersects(az)] # Only keep polygons with confirmed Aquarry points

az_qa = US_QA[US_QA.geometry.intersects(az)]
print(f'AZ QAed mines: {(az_qa['category'] == 'a').sum()}')

In [ ]:
# Hold out Miami AZ

miami_az = gpd.read_file('data/miami_az_pits.geojson')
az_mines_intersect = az_mines_intersect[~az_mines_intersect.geometry.within(miami_az.geometry.iloc[0])]

print(len(az_mines_intersect))

az_qa = az_qa[~az_qa.geometry.intersects(miami_az.geometry.iloc[0])]

#### Get MN data and separate mines / other bodies of water

In [ ]:
# Get pit lakes in Minnesota

states = gpd.read_file('data/state_boundaries')
mn = states[states['STUSPS']=='MN']
mn = mn.to_crs(epsg = 4326).geometry.unary_union # MN was multipolygon

mn_qa = US_QA[US_QA.geometry.intersects(mn)]
print(f'MN QAed mines: {(mn_qa['category'] == 'a').sum()}')

In [ ]:
# Data source: https://gisdata.mn.gov/dataset/water-dnr-hydrography

mn_water_features = gpd.read_file('data/dnr_hydro_features')
mn_water_features = mn_water_features[['wb_class','shape_Area','geometry']]
mine_classes = ['Natural Ore Mine','Mine Pit Lake','Mine Pit Lake (NF)','Tac/Natural Ore Mine']

dnr_pit_lakes = mn_water_features.loc[mn_water_features['wb_class'].isin(mine_classes)].to_crs(epsg=4326)

dnr_water = mn_water_features[~mn_water_features['wb_class'].isin(mine_classes)].to_crs(epsg=4326)

print(len(dnr_pit_lakes))
print(len(dnr_water))

dnr_pit_lakes.head()


In [ ]:
mn_qa_pits = mn_qa[mn_qa['category']=='a']
overlap_check = gpd.sjoin(mn_qa_pits, dnr_pit_lakes, how = 'inner', predicate = "intersects")
overlap_check = overlap_check.drop_duplicates(subset=["objectid"])
print(f"% of mine QA'ed data with overlapping polygon from MN DNR: {len(overlap_check)/len(mn_qa_pits)*100}")

overlap_check = gpd.sjoin(dnr_pit_lakes, mn_qa_pits, how = 'inner', predicate = "intersects")
overlap_check = overlap_check.drop_duplicates(subset=["geometry"])
print(f"% of DNR polygons with overlapping mine QA'ed data: {len(overlap_check)/len(dnr_pit_lakes)*100}")


In [ ]:
#Visualize overlap of Minnesota Aquarry QAed mines and DNR polygons
providers = cx.providers.flatten()

overlap = gpd.sjoin(dnr_pit_lakes,mn_qa_pits,how = "inner",rsuffix="1")

ax = overlap.plot(edgecolor = 'blue')
dnr_pit_lakes.plot(ax = ax,edgecolor = 'red', alpha = 0.5)
mn_qa_pits.plot(ax = ax, color = 'green',markersize = 1)

In [ ]:
# Hold out Crosby MN

crosby_mn = gpd.read_file('data/crosby_mn.geojson')
dnr_pit_lakes = dnr_pit_lakes[~dnr_pit_lakes.geometry.intersects(crosby_mn.geometry.iloc[0])]
mn_qa = mn_qa[~mn_qa.geometry.intersects(crosby_mn.geometry.iloc[0])]
dnr_water = dnr_water[~dnr_water.geometry.intersects(crosby_mn.geometry.iloc[0])]

print((mn_qa['category']=='a').sum())
print(len(dnr_water))

#### Visualize AZ and MN mines

In [ ]:
# Visualize AZ pit lakes
providers = cx.providers.flatten()

ax = az_mines_intersect.plot()
cx.add_basemap(ax, crs=az_mines_intersect.crs, source=providers['NASAGIBS.BlueMarble'])

In [ ]:
# Visualize MN pit lakes
providers = cx.providers.flatten()

ax = mn_qa.plot()
cx.add_basemap(ax, crs=mn_qa.crs, source=providers['NASAGIBS.BlueMarble'])

### Check on handling of clouds


In [ ]:
# Grab a cloudy scene 
# 7/30 is very cloudy, 7/27 has clouds in top left corner

cloudy_scene = "data/MN_v1_5/UL/2024/7/S2B_15TUL_20240730_0_L2A/S2B_15TUL_20240730_0_L2A.parquet"

cloudy_scene_chips = gpd.read_parquet(cloudy_scene)
cloudy_scene_chips = gpd.GeoDataFrame(cloudy_scene_chips).set_crs(epsg=4326)

print(len(cloudy_scene_chips))

# Perform PCA on scene
cloudy_pca = EmbeddingsPCA(cloudy_scene_chips)

In [ ]:
# Visualize scene PCA

cloudy_pca.plot(column='pca1', legend=True)

In [ ]:
# Visualize the first principal component
fig, ax = plt.subplots()

    # Plot for category 1
ax.scatter(cloudy_pca.index,
    cloudy_pca[cloudy_pca['clouds'] == True]['pca1'], 
           color='blue', alpha=0.5)

    # Plot for category 2
#ax.scatter(cloudy_pca.index,
#    cloudy_pca[cloudy_pca['clouds'] == False]['pca1'], 
#           color='grey', alpha=0.05)
    
""" EDIT SO CAN SEE WHAT THE LITTLE OVERLAP IS  plt.scatter(data[data['wb_'] == new_condition]['pca1'], 
            data[data[new_class_column] == new_condition]['pca2'], 
            color='red', label=label3, alpha=0.5, marker='^') """

plt.xlabel('chips')
plt.ylabel('PCA 1')
ax.legend()
plt.title('PCA of Cloudy Scene Embeddings')


### Grab embeddings for mines and random embeddings

In [13]:
# Define function for getting embedding files that intersect with polygons of interest
# USE WHEN DON'T HAVE BIGGER GRID ZONES TO SEARCH THROUGH

def GetEmbeddingsFromIntersection1_5(folder_path, polygons, random_images = True, count = 35):
    """ Take a folder of embeddings as a string and a set of polygons and return 1) scene file names with intersecting 
    chips and 2) a random set of non-intersecting image files
    
    Parameters:
    folder_path (str): folder with embedding gpqs
    polygons (GeoDataFrame): positive polygon masks 
    random_images (bool): True if you want random negative images
    
    Returns:
    positive_embeddings, negative embeddings (list, list): one positive, one negative set of scenes  """

    positive_embeddings = []
    negatives_list = []
    negative_embeddings = []

    # Go through all embedding folders
    # Folder structure: Grid zone, year, month, embedding folder, parquet file
    grid_zone_folders = [os.path.join(folder_path, folder) for folder in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, folder))]

    for grid_zone_folder in grid_zone_folders:
        grid_zone = os.path.basename(grid_zone_folder)
        # Get list of year folders
        year_folders = [os.path.join(grid_zone_folder, year) for year in os.listdir(grid_zone_folder) if os.path.isdir(os.path.join(grid_zone_folder, year))]
    
        for year_folder in year_folders:
            year = os.path.basename(year_folder)
            # Get list of month folders
            month_folders = [os.path.join(year_folder, month) for month in os.listdir(year_folder) if os.path.isdir(os.path.join(year_folder, month))]
            
            for month_folder in month_folders:
                month = os.path.basename(month_folder)
                year_month = f"{year}_{month}"
                    
                # Get list of embedding folders
                embedding_folders = [os.path.join(month_folder, embedding) for embedding in os.listdir(month_folder) if os.path.isdir(os.path.join(month_folder, embedding))]
                
                for embedding_folder in embedding_folders:
                    # Get all parquet files in the embedding folder
                    parquet_files = [os.path.join(embedding_folder, f) for f in os.listdir(embedding_folder) if f.endswith(".parquet")]

                    parquet_files_details = []

                    # Append parquet file details with grid zone and year_month to the list
                    for parquet_file in parquet_files:
                        parquet_files_details.append({
                            "file_path": parquet_file,
                            "grid_zone": grid_zone,
                            "year_month": year_month
                        })

                    # In each folder, go through parquet files and add file to list if it contains mines
                    # Check if image has any intersecting chips
                    for file in parquet_files_details:
                        if os.path.getsize(file['file_path']) > 0:
                           pqs = gpd.read_parquet(file['file_path'], columns=["geometry"])
                        else:
                            print(f"Skipping empty file: {file['file_path']}")
                        if not gpd.sjoin(pqs, polygons, how="inner", predicate="intersects", rsuffix="_1").empty:
                            positive_embeddings.append(file)
                        else: 
                            if random_images == True: # If there is no intersection, choose X number of random images to grab embeddings
                                negatives_list.append(file)
                                            
                    if random_images == True:
                        # Set up to get random images
                        index = np.array(random.sample(range(1, len(negatives_list)), count))

                        for i in index:
                            negative_embeddings.append(negatives_list[i])           
                        print(f'Random images: {len(negative_embeddings)}')

    return positive_embeddings, negative_embeddings

In [286]:
# Define function for getting embedding files that intersect with polygons of interest
# USE WHEN HAVE BIGGER GRID ZONES TO SEARCH THROUGH, (i.e. 16T, 16S, etc.)

def GetEmbeddingsFromIntersection1_5(folder_path, polygons, random_images = True, count = 35):
    """ Take a folder of embeddings as a string and a set of polygons and return 1) scene file names with intersecting 
    chips and 2) a random set of non-intersecting image files
    
    Parameters:
    folder_path (str): folder with embedding gpqs
    polygons (GeoDataFrame): positive polygon masks 
    random_images (bool): True if you want random negative images
    
    Returns:
    positive_embeddings, negative embeddings (list, list): one positive, one negative set of scenes  """

    positive_embeddings = []
    negatives_list = []
    negative_embeddings = []

    # Go through all embedding folders
    # Folder structure: latitude grid, Grid zone, year, month, embedding folder, parquet file
    latitude_grid_folders = [os.path.join(folder_path, folder) for folder in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, folder))]

    for latitude_grid_folder in latitude_grid_folders:
        latitude_grid = os.path.basename(latitude_grid_folder)
        # Get list of grid zone folders
        grid_zone_folders = [os.path.join(latitude_grid_folder, folder) for folder in os.listdir(latitude_grid_folder) if os.path.isdir(os.path.join(latitude_grid_folder, folder))]

        for grid_zone_folder in grid_zone_folders:
            grid_zone = os.path.basename(grid_zone_folder)
            # Get list of year folders
            year_folders = [os.path.join(grid_zone_folder, year) for year in os.listdir(grid_zone_folder) if os.path.isdir(os.path.join(grid_zone_folder, year))]
    
            for year_folder in year_folders:
                year = os.path.basename(year_folder)
                # Get list of month folders
                month_folders = [os.path.join(year_folder, month) for month in os.listdir(year_folder) if os.path.isdir(os.path.join(year_folder, month))]
                
                for month_folder in month_folders:
                    month = os.path.basename(month_folder)
                    year_month = f"{year}_{month}"
                        
                    # Get list of embedding folders
                    embedding_folders = [os.path.join(month_folder, embedding) for embedding in os.listdir(month_folder) if os.path.isdir(os.path.join(month_folder, embedding))]
                    
                    for embedding_folder in embedding_folders:
                        # Get all parquet files in the embedding folder
                        parquet_files = [os.path.join(embedding_folder, f) for f in os.listdir(embedding_folder) if f.endswith(".parquet")]

                        parquet_files_details = []

                        # Append parquet file details with grid zone and year_month to the list
                        for parquet_file in parquet_files:
                            parquet_files_details.append({
                                "file_path": parquet_file,
                                "grid_zone": grid_zone,
                                "year_month": year_month
                            })

                        # In each folder, go through parquet files and add file to list if it contains mines
                        # Check if image has any intersecting chips
                        for file in parquet_files_details:
                            if os.path.getsize(file['file_path']) > 0:
                                pqs = gpd.read_parquet(file['file_path'], columns=["geometry"])
                            else:
                                print(f"Skipping empty file: {file['file_path']}")
                            if not gpd.sjoin(pqs, polygons, how="inner", predicate="intersects", rsuffix="_1").empty:
                                positive_embeddings.append(file)
                            else: 
                                if random_images == True: # If there is no intersection, choose X number of random images to grab embeddings
                                    negatives_list.append(file)
                                                
                        if random_images == True:
                            # Set up to get random images
                            index = np.array(random.sample(range(1, len(negatives_list)), count))

                            for i in index:
                                negative_embeddings.append(negatives_list[i])           
                            print(f'Random images: {len(negative_embeddings)}')

    return positive_embeddings, negative_embeddings

In [16]:
# Get monthly aggregates of chip embeddings for a given set of embedded images (not averaging)
# Use if chip geometries overlap substantially

def MonthlyComposite(embeddings_df, polygons = None):
    """ Take a dataframe of embedding geoparquet files and get corresponding chips with < 25% overlap

    Parameters:
    embeddings_df (DataFrame): dataframe with three columns, embeddings file paths, 
    polygons (GeoDataFrame): gdf containing geometries of features of intersect. If specified, returns only chips
    that intersect these
    
    """

    monthly_agg_embeddings = gpd.GeoDataFrame()
    if polygons is not None:
        polys_geometry = polygons.unary_union

    for zone in embeddings_df['grid_zone'].unique(): # For each UTM zone
        zone_embeddings = pd.DataFrame(embeddings_df[embeddings_df['grid_zone'] == str(zone)])
        count = 1

        for index, row in zone_embeddings.iterrows(): # For each date within UTM zone
             # Will need to adjust when more months are available, currently just goes through whole list
            chip_embeddings_agg_gdf = gpd.GeoDataFrame()
            non_intersecting_chips = gpd.GeoDataFrame()
            chip_embeddings = gpd.read_parquet(row['file_path'])
            chip_embeddings = gpd.GeoDataFrame(chip_embeddings).set_crs(epsg=4326)

            if len(chip_embeddings) > 2000: # If the image is not very cloudy
                if polygons is not None: # Don't include chips that don't correspond to the polygons
                    chip_embeddings = chip_embeddings[chip_embeddings.geometry.intersects(polys_geometry)]
                if not monthly_agg_embeddings.empty:
                    # Get all chips that aren't already in the aggregation
                    chips_intersection = chip_embeddings.geometry.intersects(monthly_agg_embeddings_unified)
                    non_intersecting_chips = chip_embeddings[~chips_intersection]
                    chip_embeddings_agg_gdf = non_intersecting_chips.rename(columns={'embeddings_left': 'embeddings'})
                    
                    # Add chips that intersect with aggregation that are less than 25% overlapping
                    intersecting_chips = chip_embeddings[chips_intersection]
                    intersecting_chips['chip_area'] = intersecting_chips.geometry.area
                    intersections = intersecting_chips.geometry.intersection(monthly_agg_embeddings_unified)
                    intersecting_chips['area_overlap'] = intersections.area
                    low_intersection_mask = (intersecting_chips['area_overlap'] / intersecting_chips['chip_area']) < 0.25
                    low_intersection_chips = intersecting_chips[low_intersection_mask].rename(columns={'embeddings_left': 'embeddings'})
                    # Add chips with small overlap to aggregated embeddings
                    chip_embeddings_agg_gdf = pd.concat([chip_embeddings_agg_gdf,low_intersection_chips])
                else:
                    non_intersecting_chips = chip_embeddings
                    chip_embeddings_agg_gdf = gpd.GeoDataFrame(non_intersecting_chips['embeddings'], columns = ['embeddings'], geometry=non_intersecting_chips['geometry'])
                chip_embeddings_agg_gdf['grid_zone'] = zone
                chip_embeddings_agg_gdf['year_month'] = zone_embeddings['year_month'].iloc[0]

                # Add additional chips
                monthly_agg_embeddings = pd.concat([monthly_agg_embeddings, chip_embeddings_agg_gdf])

                # Get geometries of aggregated chips to compare against
                chip_embeddings_agg_gdf_unified = chip_embeddings_agg_gdf.unary_union
                monthly_agg_embeddings_unified = monthly_agg_embeddings.union(chip_embeddings_agg_gdf_unified).unary_union
                #count = count + 1
                print('added') 
                #if count == 3: # Check two images without clouds
                    #count = 1
                break
    
    monthly_agg_embeddings.to_crs(epsg=4326)
    monthly_agg_embeddings = monthly_agg_embeddings.drop_duplicates('geometry')

    return monthly_agg_embeddings

In [17]:
# Get embeddings by region
def RegionalData(region, zone, negatives = True):
    qa_pits = gpd.sjoin(US_QA_filtered, region, how = 'inner', predicate = 'intersects')
    qa_pits = qa_pits.dissolve(by = 'feature_idx')

    us_qa_pits = qa_pits[qa_pits['category'] == 'a']
    us_qa_nonlakes = qa_pits[~qa_pits['category'].isin(['a','q','\\N'])]

    print(f'Number of features identified as pit lakes in overlapping states in the region: {len(us_qa_pits)}')
    # Note: this number will exceed the actual amount in the region

    # Get embeddings for pit lakes and no pit lakes present in US zone from QA data
    print(f'data/US_v1_5/{zone}')
    mine_embeddings_us, non_mine_embeddings_us = GetEmbeddingsFromIntersection1_5(folder_path = f'data/US_v1_5/{zone}', polygons = us_qa_pits, random_images=False)
    print(f"Number of images with mines: {len(mine_embeddings_us)}")

    # Get monthly embeddings for pit lake embeddings in MN
    mine_embeddings_us_df = pd.DataFrame(mine_embeddings_us, columns = ['file_path','grid_zone','year_month'])

    us_positive_embs_avg = MonthlyComposite(mine_embeddings_us_df, us_qa_pits)
    print(f'Number of positive chips: {len(us_positive_embs_avg)}')
    
    us_negative_embs_avg = gpd.GeoDataFrame()
    if negatives == True:
        # Get embeddings for features identified as not lakes from QA
        qa_non_mine_embeddings_us, dummy = GetEmbeddingsFromIntersection1_5(folder_path = f'data/US_v1_5/{zone}', polygons = us_qa_nonlakes, random_images=False)
        print(f"Number of negative mine images: {len(qa_non_mine_embeddings_us)}")

        # Get monthly embeddings for water and non-pit lake embeddings in MN
        qa_non_mine_embeddings_us_df = pd.DataFrame(qa_non_mine_embeddings_us, columns = ['file_path','grid_zone','year_month'])

        us_negative_embs_avg = MonthlyComposite(qa_non_mine_embeddings_us_df, us_qa_nonlakes)
        print(f'Number of negative chips: {len(us_negative_embs_avg)}')

    return us_positive_embs_avg, us_negative_embs_avg
    


In [ ]:
# Get geometries for region 16 and 17
region16 = states_filtered[states_filtered['STUSPS'].isin(['MN', 'MI', 'IL', 'IN', 'OH', 'MS', 'AL', 'GA', 'LA', 'FL', 'MO'])]

qa_pits_16 = gpd.sjoin(US_QA_filtered, region16, how = 'inner', predicate = 'intersects')
qa_pits_16['geometry'] = qa_pits_16.geometry
qa_pits_16 = qa_pits_16.dissolve(by = 'feature_idx')

us_qa_pits_16 = qa_pits_16[qa_pits_16['category'] == 'a']
us_qa_nonlakes_16 = qa_pits_16[~qa_pits_16['category'].isin(['a','q','\\N'])]

print(len(us_qa_pits_16))

region17 = states_filtered[states_filtered['STUSPS'].isin(['MI', 'OH', 'NY', 'PA', 'WV', 'VA', 'NC', 'SC', 'KY', 'TN', 'GA','FL'])]

qa_pits_17 = gpd.sjoin(US_QA_filtered, region17, how = 'inner', predicate = 'intersects')
qa_pits_17['geometry'] = qa_pits_17.geometry
qa_pits_17 = qa_pits_17.dissolve(by = 'feature_idx')

us_qa_pits_17 = qa_pits_17[qa_pits_17['category'] == 'a']
us_qa_nonlakes_17 = qa_pits_17[~qa_pits_17['category'].isin(['a','q','\\N'])]

print(len(us_qa_pits_17))


In [19]:
# Get grid region geometries and get corresponding positive and negative embeddings from QA dataset

region16T = states_filtered[states_filtered['STUSPS'].isin(['MN', 'MI', 'IL', 'IN', 'OH'])]
region16S = states_filtered[states_filtered['STUSPS'].isin(['IL', 'IN', 'OH', 'MS', 'AL', 'GA', 'MO'])]
region16R = states_filtered[states_filtered['STUSPS'].isin(['MS', 'AL', 'GA','LA','FL'])]

us16T_positive_embs_avg, us16T_negative_embs_avg = RegionalData(region16T, '16/T')
us16R_positive_embs_avg, us16R_negative_embs_avg = RegionalData(region16R, '16/R')
us16S_positive_embs_avg, us16S_negative_embs_avg = RegionalData(region16S, '16/S')

added
added
added
added
added
added
added
added
added
added
added
added
added
111
Number of negative mine images: 711
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
62
528
data/US_v1_5/16/R
Number of images with mines: 179
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
29
Number of negative mine images: 154
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
17
656
data/US_v1_5/16/S
Number of images with mines: 636
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
a

In [ ]:
# Get grid region geometries and get corresponding positive and negative embeddings from QA dataset

region17T = states_filtered[states_filtered['STUSPS'].isin(['MI', 'OH', 'PA','NY'])]
region17S = states_filtered[states_filtered['STUSPS'].isin(['OH', 'PA', 'WV', 'VA', 'NC', 'SC', 'KY', 'TN', 'GA'])]
region17R = states_filtered[states_filtered['STUSPS'].isin(['GA','FL'])]

us17S_positive_embs_avg, us17S_negative_embs_avg = RegionalData(region17S, '17/S')
us17T_positive_embs_avg, us17T_negative_embs_avg = RegionalData(region17T, '17/T')
us17R_positive_embs_avg, us17R_negative_embs_avg = RegionalData(region17R, '17/R')

573
data/US_v1_5/17/S
Number of images with mines: 494
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
146
Number of negative mine images: 462
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
122
123
data/US_v1_5/17/T
Skipping empty file: data/US_v1_5/17/T\MG\2024\9\S2B_17TMG_20240907_0_L2A\S2B_17TMG_20240907_0_L2A.parquet
Number of images with mines: 466
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
83
Skipping empty file: data/US_v1_5/17/T\MG\2024\9\S2B

In [604]:
# Concatenate into UTM vertical zones

us16_positive_embs_avg = pd.concat([us16R_positive_embs_avg, us16S_positive_embs_avg, us16T_positive_embs_avg])
us16_negative_embs_avg = pd.concat([us16R_negative_embs_avg, us16S_negative_embs_avg, us16T_negative_embs_avg])

us17_positive_embs_avg = pd.concat([us17R_positive_embs_avg, us17S_positive_embs_avg, us17T_positive_embs_avg])
us17_negative_embs_avg = pd.concat([us17R_negative_embs_avg, us17S_negative_embs_avg, us17T_negative_embs_avg])

In [ ]:
# Visualize pit lake chips
fig, ax = plt.subplots()
us16_positive_embs_avg['geometry'].plot(ax = ax, facecolor = "none", edgecolor = "blue")
us17_positive_embs_avg['geometry'].plot(ax = ax, facecolor = "none", edgecolor = "blue")

In [ ]:
# ADD MINNESOTA DNR DATASET

In [ ]:
# Combine Aquarry QA and DNR data to check for intersection

mn_qa_pits = mn_qa[mn_qa['category'] == 'a'] # QAed pit lakes 

mn_aggregated_pits = gpd.overlay(mn_qa_pits, dnr_pit_lakes, how="union")

# Get embedding files for pit lakes in MN in DNR or Aquarry dataset
mine_embeddings_mn, dummy = GetEmbeddingsFromIntersection1_5(folder_path = 'data/MN_v1_5/', polygons = mn_aggregated_pits, random_images = False)
print(f"Number of images with mines: {len(mine_embeddings_mn)}")

mn_qa_nonlakes = mn_qa[~mn_qa['category'].isin(['a','q','\\N'])] # QAed other mining features, without questionable features 

# Get embeddings for lakes/water in MN
dnr_water = dnr_water.to_crs(epsg=4326)

# Reduce sample size
random_idx = random.sample(range(1, len(dnr_water)), 1000)
dnr_water_cropped = dnr_water.iloc[random_idx]

#dnr_water_cropped = dnr_water_cropped[~dnr_water_cropped.geometry.intersects(crosby_mn.geometry.iloc[0])] # Remove any Crosby lakes

lake_embeddings_mn, dummy = GetEmbeddingsFromIntersection1_5(folder_path = 'data/MN_v1_5/', polygons = dnr_water_cropped, random_images = False)
print(f"Number of images with water that's not mines: {len(lake_embeddings_mn)}")


# Get embeddings for features identified as not lakes from QA
qa_non_mine_embeddings_mn, dummy = GetEmbeddingsFromIntersection1_5(folder_path = 'data/MN_v1_5/', polygons = mn_qa_nonlakes, random_images = False)
print(f"Number of negative mine images: {len(qa_non_mine_embeddings_mn)}")

In [710]:
# DEPRECATED
# Make monthly aggregates
# USE IF GEOMETRIES DON'T OVERLAP TOO MUCH

def MonthlyComposite(embeddings_df):

    monthly_average_embeddings = gpd.GeoDataFrame()

    for zone in embeddings_df['grid_zone'].unique():
        zone_embeddings = pd.DataFrame(embeddings_df[embeddings_df['grid_zone'] == str(zone)])

        month_aggregate = gpd.GeoDataFrame()
        
        for index, row in zone_embeddings.iterrows():
             # Will need to adjust when more months are available
            chip_embeddings = gpd.read_parquet(row['file_path'])
            chip_embeddings = gpd.GeoDataFrame(chip_embeddings).set_crs(epsg=4326)
            month_aggregate = pd.concat([month_aggregate,chip_embeddings])

            grouped = month_aggregate.groupby('geometry')

            chip_embeddings_agg = grouped['embeddings'].apply(list).reset_index()
            chip_embeddings_agg_gdf = gpd.GeoDataFrame(chip_embeddings_agg, geometry='geometry')

            chip_embeddings_agg_gdf['monthly_average'] = chip_embeddings_agg_gdf['embeddings'].apply(
                lambda x: np.mean(np.vstack(x), axis=0))

            chip_embeddings_agg_gdf['grid_zone'] = zone
            chip_embeddings_agg_gdf['year_month'] = zone_embeddings['year_month'].iloc[0]

            monthly_average_embeddings = pd.concat([monthly_average_embeddings, chip_embeddings_agg_gdf])

    return monthly_average_embeddings


### State by state

#### Arizona

In [ ]:
# Get embeddings for pit lakes and no pit lakes present in AZ

mine_embeddings_az, non_mine_embeddings_az = GetEmbeddingsFromIntersection1_5(folder_path = 'data/AZ_v1_5/', polygons = az_mines_intersect)

print("Number of images with mines:")
print(len(mine_embeddings_az))

In [ ]:
# Get average of monthly embeddings for pit lake embeddings in AZ
mine_embeddings_az_df = pd.DataFrame(mine_embeddings_az, columns = ['file_path','grid_zone','year_month'])

az_positive_embs_avg = MonthlyComposite(mine_embeddings_az_df)
print(len(az_positive_embs_avg))

# Get average of monthly embeddings for lake and features that aren't lakes in AZ
lake_embeddings_az_df = pd.DataFrame(lake_embeddings_az, columns = ['file_path','grid_zone','year_month'])
qa_non_mine_embeddings_az_df = pd.DataFrame(qa_non_mine_embeddings_az, columns = ['file_path','grid_zone','year_month'])

az_water_embs_avg = MonthlyComposite(lake_embeddings_az_df)
az_negative_embs_avg = MonthlyComposite(qa_non_mine_embeddings_az_df)
print(len(az_water_embs_avg))
print(len(az_negative_embs_avg))

In [ ]:
#Check overlap
fig, ax = plt.subplots()
az_positive_embs_avg['geometry'].plot(ax = ax, facecolor = "none", edgecolor = "blue")

#az_negative_embs_avg['geometry'].plot(ax = ax, facecolor = "none", edgecolor = "blue")


In [ ]:
# Get embeddings for pit lakes and no pit lakes present in AZ from QA data

az_qa_pits = az_qa[az_qa['category'] == 'a'] # QAed pit lakes 
az_qa_nonlakes = az_qa[~az_qa['category'].isin(['a','q','\\N'])] # QAed other mining features, without questionable features 

mine_embeddings_az, non_mine_embeddings_az = GetEmbeddingsFromIntersection1_5(folder_path = 'data/AZ_v1_5/', polygons = az_qa_pits, random_images=False)
print(f"Number of images with mines: {len(mine_embeddings_az)}")


# Get embeddings for features identified as not lakes from QA
qa_non_mine_embeddings_az, dummy = GetEmbeddingsFromIntersection1_5(folder_path = 'data/AZ_v1_5/', polygons = az_qa_nonlakes, random_images=False)
print(f"Number of negative mine images: {len(qa_non_mine_embeddings_az)}")


# Get embeddings for water bodies in AZ
az_lakes = gpd.read_file('data/az_lakes.txt')
lake_embeddings_az, dummy = GetEmbeddingsFromIntersection1_5(folder_path = 'data/AZ_v1_5/', polygons = az_lakes, random_images=False)

print(f"Number of images with lakes: {len(lake_embeddings_az)}")

#### Indiana

In [ ]:
# Get pit lakes in IN

states = gpd.read_file('data/state_boundaries')
indy = states[states['STUSPS']=='IN']
indy = indy.to_crs(epsg = 4326).geometry.unary_union

in_qa = US_QA[US_QA.geometry.intersects(indy)]
print(f'IN QAed mines: {(in_qa['category'] == 'a').sum()}')

In [ ]:
# Get embeddings for pit lakes and no pit lakes present in IN from QA data

in_qa_pits = in_qa[in_qa['category'] == 'a'] # QAed pit lakes 
in_qa_nonlakes = in_qa[~in_qa['category'].isin(['a','q','\\N'])] # QAed other mining features, without questionable features 

mine_embeddings_in, non_mine_embeddings_in = GetEmbeddingsFromIntersection1_5(folder_path = 'data/IN_v1_5/', polygons = in_qa_pits, random_images=False)
print(f"Number of images with mines: {len(mine_embeddings_in)}")


# Get embeddings for features identified as not lakes from QA
qa_non_mine_embeddings_in, dummy = GetEmbeddingsFromIntersection1_5(folder_path = 'data/IN_v1_5/', polygons = in_qa_nonlakes, random_images=False)
print(f"Number of negative mine images: {len(qa_non_mine_embeddings_in)}")

# Get embeddings for water bodies in IN
#az_lakes = gpd.read_file('data/az_lakes.txt')
#lake_embeddings_az, dummy = GetEmbeddingsFromIntersection1_5(folder_path = 'data/IN_v1_5/', polygons = az_lakes, random_images=False)

#print(f"Number of images with lakes: {len(lake_embeddings_az)}")

In [ ]:
# Get average of monthly embeddings for pit lake embeddings in IN
mine_embeddings_in_df = pd.DataFrame(mine_embeddings_in, columns = ['file_path','grid_zone','year_month'])

in_positive_embs_avg = MonthlyComposite(mine_embeddings_in_df)
print(len(in_positive_embs_avg))

# Get average of monthly embeddings for lake and features that aren't lakes in AZ
#lake_embeddings_in_df = pd.DataFrame(lake_embeddings_in, columns = ['file_path','grid_zone','year_month'])
qa_non_mine_embeddings_in_df = pd.DataFrame(qa_non_mine_embeddings_in, columns = ['file_path','grid_zone','year_month'])

#az_water_embs_avg = MonthlyComposite(lake_embeddings_in_df)
in_negative_embs_avg = MonthlyComposite(qa_non_mine_embeddings_in_df)
#print(len(in_water_embs_avg))
print(len(in_negative_embs_avg))

In [ ]:
#Check mines chips
fig, ax = plt.subplots()
in_positive_embs_avg['geometry'].plot(ax = ax, facecolor = "none", edgecolor = "blue")


#### Kentucky

In [ ]:
# Get embeddings for pit lakes and no pit lakes present in IN from QA data

ky_qa_pits = ky_qa[ky_qa['category'] == 'a'] # QAed pit lakes 
ky_qa_nonlakes = ky_qa[~ky_qa['category'].isin(['a','q','\\N'])] # QAed other mining features, without questionable features 

mine_embeddings_ky, non_mine_embeddings_in = GetEmbeddingsFromIntersection1_5(folder_path = 'data/KY_v1_5/', polygons = ky_qa_pits, random_images=False)
print(f"Number of images with mines: {len(mine_embeddings_ky)}")

# Get embeddings for features identified as not lakes from QA
qa_non_mine_embeddings_ky, dummy = GetEmbeddingsFromIntersection1_5(folder_path = 'data/KY_v1_5/', polygons = ky_qa_nonlakes, random_images=False)
print(f"Number of negative mine images: {len(qa_non_mine_embeddings_ky)}")

# Get embeddings for water bodies in IN
#az_lakes = gpd.read_file('data/az_lakes.txt')
#lake_embeddings_az, dummy = GetEmbeddingsFromIntersection1_5(folder_path = 'data/IN_v1_5/', polygons = az_lakes, random_images=False)

#print(f"Number of images with lakes: {len(lake_embeddings_az)}")

In [ ]:
# Get average of monthly embeddings for pit lake embeddings in IN
mine_embeddings_ky_df = pd.DataFrame(mine_embeddings_ky, columns = ['file_path','grid_zone','year_month'])

ky_positive_embs_avg = MonthlyComposite(mine_embeddings_ky_df)
print(len(ky_positive_embs_avg))

# Get average of monthly embeddings for lake and features that aren't lakes in AZ
#lake_embeddings_in_df = pd.DataFrame(lake_embeddings_in, columns = ['file_path','grid_zone','year_month'])
qa_non_mine_embeddings_ky_df = pd.DataFrame(qa_non_mine_embeddings_ky, columns = ['file_path','grid_zone','year_month'])

#az_water_embs_avg = MonthlyComposite(lake_embeddings_in_df)
ky_negative_embs_avg = MonthlyComposite(qa_non_mine_embeddings_ky_df)
#print(len(in_water_embs_avg))
print(len(ky_negative_embs_avg))

In [ ]:
#Check mines chips
fig, ax = plt.subplots()
ky_positive_embs_avg['geometry'].plot(ax = ax, facecolor = "none", edgecolor = "blue")

#### Minnesota

In [ ]:
# Get monthly embeddings for pit lake embeddings in MN
mine_embeddings_mn_df = pd.DataFrame(mine_embeddings_mn, columns = ['file_path','grid_zone','year_month'])

mn_positive_embs_avg = MonthlyComposite(mine_embeddings_mn_df, mn_aggregated_pits)
print(len(mn_positive_embs_avg))

# Get monthly embeddings for water and non-pit lake embeddings in MN
lake_embeddings_mn_df = pd.DataFrame(lake_embeddings_mn, columns = ['file_path','grid_zone','year_month'])
qa_non_mine_embeddings_mn_df = pd.DataFrame(qa_non_mine_embeddings_mn, columns = ['file_path','grid_zone','year_month'])

mn_water_embs_avg = MonthlyComposite(lake_embeddings_mn_df, dnr_water_cropped)
mn_negative_embs_avg = MonthlyComposite(qa_non_mine_embeddings_mn_df, mn_qa_nonlakes)
print(len(mn_water_embs_avg))
print(len(mn_negative_embs_avg))

In [ ]:
#Check overlap
fig, ax = plt.subplots()
mn_positive_embs_avg['geometry'].plot(ax = ax, facecolor = "none", edgecolor = "blue")


In [ ]:
# Get embeddings for pit lakes in MN in DNR dataset
mine_embeddings_mn, dummy = GetEmbeddingsFromIntersection(folder_path = 'data/60cm_mn/rgbir_cog/mn/', polygons = dnr_pit_lakes, random_images = False)

print(f"Number of images with mines: {len(mine_embeddings_mn)}")


### Label embeddings with mine/no mine

In [929]:
# Define function that marks each chip in intersecting and non-intersecting set of images as True if intersects chip embedding, False if not

def LabelEmbeddings(positive_embeddings, negative_embeddings, polygons = gpd.GeoDataFrame(), polygons2 = gpd.GeoDataFrame()):
    """ Create GeoDataFrame that takes image files with intersection with polygon of interest and returns only positive chips and marks True and 
    takes images without intersection and mark all chips as False 
    
    Helpful if there may be false negatives within the image with intersecting polygons
    
    Parameters:
    positive_embeddings (GeoDataFrame): positive mask files
    negative_embeddings (GeoDataFrame): negative mask files

    Returns:
    data (GeoDataFrame): chips with positive and negative labels

    """
    data = gpd.GeoDataFrame()

    for emb in positive_embeddings:
        chip_embeddings = gpd.read_parquet(emb)
        chip_embeddings = gpd.GeoDataFrame(chip_embeddings).set_crs(epsg=4326)
        positive_chips = gpd.sjoin(chip_embeddings, polygons, predicate='intersects',how = 'inner',rsuffix="_1")
        data = pd.concat([data, positive_chips], ignore_index=True)

    data['mine'] = 1 # Assign 1 to mine column for chips with embeddings
    print("Number of positive samples:") 
    print(len(data))

    if polygons2.empty:
        for emb in negative_embeddings:
            chip_embeddings = gpd.read_parquet(emb)
            chip_embeddings = gpd.GeoDataFrame(chip_embeddings).set_crs(epsg=4326)
            chip_embeddings['mine'] = 0 # Assign 0 to mine column for chips without embeddings
            data = pd.concat([data, chip_embeddings], ignore_index=True)
    else:
        for emb in negative_embeddings:
            chip_embeddings = gpd.read_parquet(emb)
            chip_embeddings = gpd.GeoDataFrame(chip_embeddings).set_crs(epsg=4326)
            negative_chips = gpd.sjoin(chip_embeddings, polygons2, predicate='intersects',how = 'inner',rsuffix="_1")
            negative_chips['mine'] = 0
            data = pd.concat([data, negative_chips], ignore_index=True)

    return data

In [141]:
# Define function that marks each chip in intersecting and non-intersecting set of images as True if intersects chip embedding, False if not

def LabelEmbeddings1_5(positive_embeddings, negative_embeddings, polygons = gpd.GeoDataFrame(), polygons2 = gpd.GeoDataFrame()):
    """ Create GeoDataFrame that takes image files with intersection with polygon of interest and returns only positive chips and marks True and 
    takes images without intersection and mark all chips as False or returns only chips that intersect with polygons2
    
    Helpful if there may be false negatives within the image with intersecting polygons
    
    Parameters:
    positive_embeddings (GeoDataFrame): positive chip embeddings
    negative_embeddings (GeoDataFrame): negative chip embeddings

    Returns:
    data (GeoDataFrame): chips with positive and negative labels

    """
    data = gpd.GeoDataFrame()

    positive_chips = gpd.sjoin(positive_embeddings, polygons, predicate='intersects',how = 'inner',rsuffix="_1")
    positive_chips = positive_chips[~positive_chips['index_right'].isna()]
    print(len(positive_chips))

    data = pd.concat([data, positive_chips], ignore_index=True)
    data['mine'] = 1 # Assign 1 to mine column for chips with embeddings

    if polygons2.empty:
        for emb in negative_embeddings:
            negative_embeddings['mine'] = 0 # Assign 0 to mine column for chips without overlap
            data = pd.concat([data, negative_embeddings], ignore_index=True)
    else:
        for emb in negative_embeddings:
            negative_chips = gpd.sjoin(negative_embeddings, polygons2, predicate='intersects',how = 'inner',rsuffix="_1")
            negative_chips = negative_chips[~negative_chips['index_right'].isna()]
            negative_chips['mine'] = 0
            data = pd.concat([data, negative_chips], ignore_index=True)

    # If chips have both pit lakes and lakes, they will appear multiple times in data. 
    # Mark chip as positive if it has at least one pit lake, and drop duplicates.
    data['mine'] = data.groupby('geometry')['mine'].transform('max')  

    # Drop duplicates if you want to keep only one row per index
    data = data.drop_duplicates(subset=['geometry'])
    print(f'Number of positive samples: {(data['mine'] == 1).sum()}') 

    return data

In [ ]:
data16R = LabelEmbeddings1_5(us16R_positive_embs_avg, us16R_negative_embs_avg, us_qa_pits_16, us_qa_nonlakes_16)
data16S = LabelEmbeddings1_5(us16S_positive_embs_avg, us16S_negative_embs_avg, us_qa_pits_16, us_qa_nonlakes_16)
data16T = LabelEmbeddings1_5(us16T_positive_embs_avg, us16T_negative_embs_avg, us_qa_pits_16, us_qa_nonlakes_16)

print((data16R['mine'] == 1).sum())
print((data16S['mine'] == 1).sum())
print((data16T['mine'] == 1).sum())

In [ ]:
data17R = LabelEmbeddings1_5(us17R_positive_embs_avg, us17R_negative_embs_avg, us_qa_pits_17, us_qa_nonlakes_17)
data17S = LabelEmbeddings1_5(us17S_positive_embs_avg, us17S_negative_embs_avg, us_qa_pits_17, us_qa_nonlakes_17)
data17T = LabelEmbeddings1_5(us17T_positive_embs_avg, us17T_negative_embs_avg, us_qa_pits_17, us_qa_nonlakes_17)

print((data17R['mine'] == 1).sum())
print((data17S['mine'] == 1).sum())
print((data17T['mine'] == 1).sum())

### State by state approach

#### Arizona

In [ ]:
# Get chips with mines and no mines in AZ from QA data
az_data = LabelEmbeddings(mine_embeddings_az, non_mine_embeddings_az, az_qa_lakes)
print(f"Total samples: {len(az_data)}") 


# Get chips with lakes in AZ
az_lakes_chips = LabelEmbeddings(dummy, lake_embeddings_az, gpd.GeoDataFrame(), az_lakes)
print(f"Total samples: {len(az_lakes_chips)}") 

az_data = pd.concat([az_data,az_lakes_chips], ignore_index=True) # Add lakes to AZ embeddings


# Get chips for features identified as not lakes from QA
az_nonlakes_chips = LabelEmbeddings(dummy, qa_non_mine_embeddings_az, gpd.GeoDataFrame(), az_qa_nonlakes)
print(f"Total samples: {len(az_nonlakes_chips)}") 

az_data = pd.concat([az_data,az_nonlakes_chips], ignore_index=True) # Add not pit lakes to AZ embeddings

In [ ]:
# Get chips with pit lakes and non pit lakes in AZ 
az_data = LabelEmbeddings1_5(az_positive_embs_avg, az_negative_embs_avg, az_qa_pits, az_qa_nonlakes)

print("Total samples:") 
print(len(az_data))

# Get chips with regular bodies of eater
az_add_data = LabelEmbeddings1_5(az_positive_embs_avg, az_water_embs_avg, dnr_pit_lakes, az_lakes)

non_mines = az_add_data[az_add_data['mine'] == 0]

az_data = pd.concat([az_data, non_mines])
az_data = az_data.drop_duplicates('geometry')

print("Total samples:") 
print(len(az_data))

In [ ]:
# Get chips with mines and no mines in AZ
az_data = LabelEmbeddings(mine_embeddings_az, non_mine_embeddings_az, az_mines_intersect)

print("Total samples:") 
print(len(az_data))

#### Indiana and KY

In [ ]:
# Get chips with pit lakes and non pit lakes in IN

in_data = LabelEmbeddings1_5(in_positive_embs_avg, in_negative_embs_avg, in_qa_pits, in_qa_nonlakes)

print("Total samples:") 
print(len(in_data))

# Get chips with regular bodies of eater
#in_add_data = LabelEmbeddings1_5(az_positive_embs_avg, az_water_embs_avg, dnr_pit_lakes, az_lakes)

#non_mines = az_add_data[az_add_data['mine'] == 0]
#in_data = pd.concat([in_data, non_mines])

in_data = in_data.drop_duplicates('geometry')

print("Total samples:") 
print(len(in_data))

In [ ]:
# Get chips with pit lakes and non pit lakes in KY 

ky_data = LabelEmbeddings1_5(ky_positive_embs_avg, ky_negative_embs_avg, ky_qa_pits, ky_qa_nonlakes)

print("Total samples:") 
print(len(ky_data))

# Get chips with regular bodies of eater
#in_add_data = LabelEmbeddings1_5(az_positive_embs_avg, az_water_embs_avg, dnr_pit_lakes, az_lakes)

#non_mines = az_add_data[az_add_data['mine'] == 0]
#in_data = pd.concat([in_data, non_mines])

ky_data = ky_data.drop_duplicates('geometry')

print("Total samples:") 
print(len(ky_data))

In [ ]:
# Get chips with pit lakes and regular bodies of water in MN in DNR dataset
mn_data = LabelEmbeddings1_5(mn_positive_embs_avg, mn_water_embs_avg, dnr_pit_lakes, dnr_water_cropped)

print("Total samples:") 
print(len(mn_data))

# Add QA'ed non lakes
mn_add_data = LabelEmbeddings1_5(mn_positive_embs_avg, mn_negative_embs_avg, dnr_pit_lakes, mn_qa_nonlakes)

non_mines = mn_add_data[mn_add_data['mine'] == 0]

mn_data = pd.concat([mn_data, non_mines])
print("Total samples:") 
print(len(mn_data))

# Add in pit lakes that are only in the Aquarry QA dataset. Not removing the extra land area for now
intersecting_aquarry_dnr = gpd.sjoin(mn_qa_pits, dnr_pit_lakes, how='left', predicate='intersects').drop_duplicates('geometry')
aquarry_only_pits = mn_qa_pits[intersecting_aquarry_dnr['index_right'].isna()]
print(f'Pits only in QA dataset: {len(aquarry_only_pits)}')

mn_add_data = LabelEmbeddings1_5(mn_positive_embs_avg, mn_negative_embs_avg, aquarry_only_pits, mn_qa_nonlakes)

aquarry_mn_mines = mn_add_data[mn_add_data['mine'] == 1]

mn_data = pd.concat([mn_data, aquarry_mn_mines])
mn_data = mn_data.drop_duplicates('geometry')

print("Total samples:") 
print(len(mn_data))

In [ ]:
mn_data.to_crs(epsg=32615).iloc[0]

### Rasterize polygons

In [33]:
# Define function that assigns corresponding polygons to each chip
from shapely.ops import unary_union

def GetPolygonsforChips(embeddings, pit_lakes):
    """Get intersecting pit lakes for each embedding chip geometry
    """
    embeddings['pit_lake_poly'] = None
    embeddings = embeddings.reset_index(drop=True)

    for idx, chip in embeddings.iterrows():
            intersecting = pit_lakes[pit_lakes.intersects(chip['geometry'])]
            embeddings.at[idx, 'pit_lake_poly'] = intersecting['geometry'].tolist()     

    embeddings = embeddings[embeddings['pit_lake_poly'].apply(lambda x: len(x) > 0)] # drop empty rows
    embeddings['pit_lake_poly'] = embeddings['pit_lake_poly'].apply(lambda x: unary_union(x)) # Get multipolygons from a list
    embeddings['pit_lake_poly'] = gpd.GeoSeries(embeddings['pit_lake_poly'], crs = 4326).to_crs(epsg=32615) # Convert geometries to epsg:32615
    embeddings = embeddings.to_crs(epsg=32615) # Convert geometries to epsg:32615

    return embeddings

In [34]:
# Define function to take chip embeddings with corresponding polygons and rasterize
from rasterio.features import rasterize
from shapely.geometry import Polygon

def RasterizePolygons(data_with_polys):
    """ Get rasterized polygons

    Parameters:
    data_with_polys (GeoDataFrame): Geodataframe with embedding geometries as geometry 
    and with multipolygons of pit lakes that intersect with each chip

    Outputs:
    rasters (list): list of numpy raster arrays for each chip
    """
    
    rasters = []

    for idx, chip in data_with_polys.iterrows():

        # Define raster dimensions based on chip's bounding box
        bounds = chip.geometry.bounds

        resolution = 10
        width = int((bounds[2] - bounds[0]) / resolution)
        height = int((bounds[3] - bounds[1]) / resolution)
        
        # Define raster transform
        transform = rasterio.transform.from_bounds(*bounds, width, height)
        
        # Create a list of geometries for rasterization
        geometries = [chip['pit_lake_poly']]
        print(geometries)
        print(height, width)
        
        # Rasterize the pit lakes
        chip_raster = rasterize(
            geometries,
            out_shape=(height, width),
            transform=transform,
            fill=0,
            dtype='uint8'
        )

        rasters.append(chip_raster)

    return rasters

In [35]:
def Createxarray(rasters, embs_with_polys):
    """Convert list of chip rasters to xarrays
    """
    rasters_xr = []

    for i in range(len(rasters)):
        lon_min, lat_min, lon_max, lat_max = embs_with_polys.geometry[i].bounds

        # Generate latitude and longitude arrays
        latitudes = np.linspace(lat_max, lat_min, rasters[i].shape[0])
        longitudes = np.linspace(lon_min, lon_max, rasters[i].shape[1])

        # Create xarray DataArray
        data_array = xr.DataArray(
            rasters[i],
            dims=["lat", "lon"],
            coords={"lat": latitudes, "lon": longitudes},
            name="pit_lakes" 
        )

        # Create xarray Dataset
        dataset = xr.Dataset({"pit_lakes": data_array})
        rasters_xr.append(dataset)

    return rasters_xr

In [594]:
# Minnesota
chips_with_polygons = mn_data[mn_data['mine']==1] # Ensure this is right mn_data
chips_with_polygons = chips_with_polygons.drop_duplicates('geometry')
mn_data_with_polys = GetPolygonsforChips(chips_with_polygons, dnr_pit_lakes)

embs_with_polys = pd.concat([mn_data_with_polys,az_data_with_polys]).reset_index(drop=True)

# Aquarry mine testing
chips_with_polygons = aquarry_mn_mines.drop_duplicates('geometry')
aq_data_with_polys = GetPolygonsforChips(chips_with_polygons, aquarry_only_pits)
embs_with_polys = aq_data_with_polys
rasters = RasterizePolygons(embs_with_polys)
rasters_xr = Createxarray(rasters, embs_with_polys)

In [ ]:
import xarray as xr

# Get intersecting pit lakes and rasterize
us_data_mines16 = pd.concat([data16R, data16S, data16T]).drop_duplicates('geometry')

us_chips_with_polygons16 = us_data_mines16
us_embs_with_polys16 = GetPolygonsforChips(us_chips_with_polygons16, us_qa_pits_16) # drops rows without intersection
us_rasters16 = RasterizePolygons(us_embs_with_polys16)
us_embs_with_polys16 = us_embs_with_polys16.reset_index(drop=True)
us_rasters_xr16 = Createxarray(us_rasters16, us_embs_with_polys16) # Uncomment when running normally

In [ ]:
# Get intersecting pit lakes and rasterize

us_data_mines17 = pd.concat([data17R, data17S, data17T])

us_chips_with_polygons17 = us_data_mines17.drop_duplicates('geometry')
us_embs_with_polys17 = GetPolygonsforChips(us_chips_with_polygons17, us_qa_pits_17) # drops empty rows
us_rasters17 = RasterizePolygons(us_embs_with_polys17)
us_embs_with_polys17 = us_embs_with_polys17.reset_index(drop=True)
us_rasters_xr17 = Createxarray(us_rasters17, us_embs_with_polys17) # Uncomment when running normally

### NDWI Filtering

In [37]:
# Optimize GDAL settings for cloud optimized reading
def SearchSTAC(aoi, start, end):
    os.environ["GDAL_DISABLE_READDIR_ON_OPEN"] = "EMPTY_DIR"
    os.environ["AWS_REQUEST_PAYER"] = "requester"

    STAC_API = "https://earth-search.aws.element84.com/v1"
    COLLECTION = "sentinel-2-l2a"

    # Search the catalogue
    catalog = pystac_client.Client.open(STAC_API)
    search = catalog.search(
        collections=[COLLECTION],
        datetime=f"{start}/{end}",
        intersects=aoi,
        #max_items=,
        query={"eo:cloud_cover": {"lt": 10}},
    )

    all_items = search.get_all_items()
    print(len(all_items))

    # Reduce to one per date (there might be some duplicates
    # based on the location)
    items = []
    dates = []
    for item in all_items:
        if item.datetime.date() not in dates:
            items.append(item)
            dates.append(item.datetime.date())

    print(f"Found {len(items)} items")
    return items, dates

In [39]:
# Retrieve the pixel values for the bounding box in the target projection.
def RetrievePixels(items,aoi_bounds):
    dask.config.set({"array.slicing.split_large_chunks": True})
    stack = None

    if len(items) != 0:
        epsg = items[0].properties["proj:epsg"]
        gsd = 10
        valid_stacks = []

        bounds_gdf = gpd.GeoDataFrame(geometry=[aoi_bounds], crs='EPSG:32615')
        #bounds_gdf = bounds_gdf.to_crs(epsg=epsg)
        transformed_bbox = bounds_gdf.geometry[0].bounds

    for item in items:
        try:
            # Attempt to create the stack for the current item
            stack = stackstac.stack(
                [item],  # Process one item at a time
                snap_bounds=True,
                epsg=32615,
                resolution=gsd,
                dtype="float32",
                rescale=False,
                fill_value=np.float32(0),
                assets=["green", "nir"],
                resampling=Resampling.nearest,
                chunksize=512,
                bounds=transformed_bbox
            )
            valid_stacks.append(stack)
        except RuntimeError as e:
            # Log and skip the problematic item
            print(f'Error creating stack for item {item.id}: {e}')
            continue

        if len(valid_stacks) != 0:
            combined_stack = dask.array.concatenate(valid_stacks, axis=0).compute()
            stack = stack.compute()
        
    return stack

In [40]:
# Calculate NDWI and create water mask 
def CalculateNDWI(image):
    ndwi = image
    green = image.sel(band="green")
    nir = image.sel(band="nir")

    ndwi = (green - nir) / (green + nir)
    return ndwi

In [41]:
def FilterRasters(rasters, embs_with_polys):
    # Date range
    start = "2024-07-01"    
    end = "2024-09-15"

    for i in range(len(rasters)):
        
        #Get NDWI
        aoi_epsg4326 = embs_with_polys['geometry'].to_crs(epsg=4326)[i] 
        bounds = embs_with_polys['geometry'][i]    # Should be in epsg:32615

        ndwiimages, ndwidates = SearchSTAC(aoi_epsg4326, start, end) 

        try: 
            ndwistack = RetrievePixels(ndwiimages, bounds)
            summer_composite = ndwistack.groupby('band').median(dim = 'time')
            ndwi_summer = CalculateNDWI(summer_composite)

            # Compute mask
            water_mask = xr.where(ndwi_summer > 0.1, 1, 0)

            # Account for small differences
            # Create new evenly spaced coordinates and interploate
            if water_mask.x.shape[0] != water_mask.y.shape[0]:
                new_coords = np.linspace(water_mask.coords['x'][0], water_mask.coords['x'][-2], water_mask.x.shape[0])

                water_mask_resampled = water_mask.interp(x=new_coords)
            else:
                water_mask_resampled = water_mask

            # Rename coords to match
            water_mask_resampled = water_mask_resampled.rename({'y': 'lat', 'x': 'lon'})
            print(water_mask_resampled.sum().values)
            # Resample to make water mask match 

            water_mask_resampled = water_mask_resampled.interp(lat=rasters[i]['pit_lakes']['lat'].values,lon=rasters[i]['pit_lakes']['lon'].values, method='nearest')

            print(water_mask_resampled.sum().values)
    
            mask = (rasters[i]['pit_lakes'] == 1).compute()
            print(mask.sum().values)

            rasters[i]['filtered_pit_lakes'] = rasters[i]['pit_lakes']
            rasters[i]['filtered_pit_lakes'] = xr.where(mask, water_mask_resampled, rasters[i]['pit_lakes'])
            print(rasters[i]['filtered_pit_lakes'].shape)
            print(rasters[i]['filtered_pit_lakes'].sum().values)

        except RuntimeError as e:
            print(f'Error filtering raster {i}')
            continue
        
        except AttributeError:
            print(f'No images found for {i}')
            continue

    return rasters

In [ ]:
# Get filtered polygons for US 16
us_results16 = FilterRasters(us_rasters_xr16, us_embs_with_polys16)

In [ ]:
# Get filtered polygons for US 17
us_results17 = FilterRasters(us_rasters_xr17, us_embs_with_polys17)

In [ ]:
#Attempt to store, unfinished

combined1 = xr.concat(us_results[0:150], dim="band")  # Combine along a new dimension "band"
combined1 = combined1.assign_coords(band=range(1, 150 + 1))  # Assign band numbers

# Write to a multi-band GeoTIFF
combined1.rio.write_crs("EPSG:32615", inplace=True)  # Set CRS
combined1.rio.to_raster("data/us_rasters_xr16_1.tif")

combined2 = xr.concat(us_results[150:300], dim="band")  # Combine along a new dimension "band"
combined2 = combined2.assign_coords(band=range(151, 300 + 1))  # Assign band numbers

# Write to a multi-band GeoTIFF
combined2.rio.write_crs("EPSG:32615", inplace=True)  # Set CRS
combined2.rio.to_raster("data/us_rasters_xr16_2.tif")


combined1 = xr.concat(us_results17[0:130], dim="band")  # Combine along a new dimension "band"
combined1 = combined1.assign_coords(band=range(1, 130 + 1))  # Assign band numbers
combined1.rio.write_crs("EPSG:32615", inplace=True)  # Set CRS
combined1.rio.to_raster("data/us_rasters_xr17_1.tif")

combined2 = xr.concat(us_results17[130:267], dim="band")  # Combine along a new dimension "band"
combined2 = combined2.assign_coords(band=range(131, 267 + 1))  # Assign band numbers
combined2.rio.write_crs("EPSG:32615", inplace=True)  # Set CRS
combined2.rio.to_raster("data/us_rasters_xr17_2.tif")



In [982]:
# Visually inspect NDWI filters
for i in range(0,4):
    #us_results[i] = us_results[i].rename({'lon': 'x', 'lat': 'y'})
    us_results16[i]['filtered_pit_lakes'].rio.write_crs("EPSG:32615", inplace=True)  # Set CRS
    us_results16[i]['filtered_pit_lakes'].rio.to_raster(f"data/filtered_lakes_{i + 1}.tif")

In [573]:
# Resample to 256 x 256
def TrainingResample(rasters):
    rasters_list = []

    for i in range(len(rasters)):
        lat_min = float(rasters[i]['filtered_pit_lakes']['lat'].min().values)
        lat_max = float(rasters[i]['filtered_pit_lakes']['lat'].max().values)
        lon_min = float(rasters[i]['filtered_pit_lakes']['lon'].min().values)
        lon_max = float(rasters[i]['filtered_pit_lakes']['lon'].max().values)
        new_lat = np.linspace(lat_min, lat_max, 256)
        new_lon = np.linspace(lon_min, lon_max, 256)

        # Interpolate to the new grid
        resampled = rasters[i]['filtered_pit_lakes'].interp(lat=new_lat, lon=new_lon, method='nearest')
        rasters_list.append(resampled)

    return rasters_list

In [ ]:
# DELETE?
# Get only those with results and resize
filtered_results16 = [xr for xr in us_results16 if 'filtered_pit_lakes' in xr.data_vars] # If images were found
filtered_results16 = [arr for arr in filtered_results16 if arr['filtered_pit_lakes'].values.any()] # If array is non 0

resampled_us = TrainingResample(filtered_results)
resampled_us[0].plot()

# Get only those with results and resize
filtered_results17 = [xr for xr in us_results17 if 'filtered_pit_lakes' in xr.data_vars] # If images were found
filtered_results17 = [arr for arr in filtered_results17 if arr['filtered_pit_lakes'].values.any()] # If array is non 0

resampled_us17 = TrainingResample(filtered_results)
resampled_us17[0].plot()

In [ ]:
# Drop any rasters without any pit lake intersections
final_indices = []
us_embs_with_polys_reset16 = us_embs_with_polys16.reset_index(drop=True) 

# Check for rasters without pit lakes
for i, xr in enumerate(us_results16):
    # Step 1: Check if 'filtered_pit_lakes' exists in data_vars
    if 'filtered_pit_lakes' in xr.data_vars:
        # Step 2: Check if 'filtered_pit_lakes' has non-zero values
        if xr['filtered_pit_lakes'].values.any():
            final_indices.append(i)

us_results16_clean = [xr for i, xr in enumerate(us_results16) if i in final_indices]

resampled_us16 = TrainingResample(us_results16_clean)

# Drop rows in us_embs_with_polys not corresponding to final indices
us_embs_with_polys_dropped16 = us_embs_with_polys_reset16.loc[final_indices]
print(len(us_embs_with_polys_dropped16))
print(len(resampled_us16))

In [ ]:
# Drop any rasters without any pit lake intersections
final_indices = []
us_embs_with_polys_reset17 = us_embs_with_polys17.reset_index(drop=True) 

# Check for rasters without pit lakes
for i, xr in enumerate(us_results17):
    # Step 1: Check if 'filtered_pit_lakes' exists in data_vars
    if 'filtered_pit_lakes' in xr.data_vars:
        # Step 2: Check if 'filtered_pit_lakes' has non-zero values
        if xr['filtered_pit_lakes'].values.any():
            final_indices.append(i)

us_results17_clean = [xr for i, xr in enumerate(us_results17) if i in final_indices]

resampled_us17 = TrainingResample(us_results17_clean)

# Drop rows in us_embs_with_polys not corresponding to final indices
us_embs_with_polys_dropped17 = us_embs_with_polys_reset17.loc[final_indices]
print(len(us_embs_with_polys_dropped17))
print(len(resampled_us17))

In [ ]:
# Separate into seasonal
q1stack, q1dates = SearchSTAC(aoi,"2024-01-01", "2024-03-31")
q2stack, q2dates = SearchSTAC(aoi,"2024-04-01", "2024-06-30")
q3stack, q3dates = SearchSTAC(aoi,"2024-07-01", "2024-09-30")
q4stack, q4dates = SearchSTAC(aoi,"2024-10-01", "2024-12-31")

stack1 = RetrievePixels(q1stack)
stack2 = RetrievePixels(q2stack)
stack3 = RetrievePixels(q3stack)
stack4 = RetrievePixels(q4stack)

# Get median of values across time for different bands for each season
import xarray as xr

q1_composite = stack1.groupby('band').median(dim = 'time')
q2_composite = stack2.groupby('band').median(dim = 'time')
q3_composite = stack3.groupby('band').median(dim = 'time')
q4_composite = stack4.groupby('band').median(dim = 'time')


ndwi_q1 = CalculateNDWI(q1_composite)
ndwi_q2 = CalculateNDWI(q2_composite)
ndwi_q3 = CalculateNDWI(q3_composite)
ndwi_q4 = CalculateNDWI(q4_composite)

ndwi_ds = [ndwi_q1, ndwi_q2, ndwi_q3, ndwi_q4]

for i in range(len(ndwi_ds)):
    if 's2:processing_baseline' in ndwi_ds[i].coords:
        ndwi_ds[i] = ndwi_ds[i].reset_coords('s2:processing_baseline', drop=True)
        
annual_median = xr.concat(ndwi_ds, dim="time").median(dim='time')

### Train a CNN to predict pixel masks

In [486]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EmbeddingToRasterCNN(nn.Module):
    def __init__(self, embedding_dim=1024, output_size=(256, 256)):
        super(EmbeddingToRasterCNN, self).__init__()
        self.embedding_dim = embedding_dim
        self.output_size = output_size

        # Fully connected layer to project embedding to a smaller feature map
        self.fc = nn.Linear(embedding_dim, 8 * 8 * 128)  # 8x8 feature map with 128 channels

        # Transpose convolution layers to upsample the feature map progressively
        self.deconv1 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)  # 8x8 -> 16x16
        self.deconv2 = nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1)   # 16x16 -> 32x32
        self.deconv3 = nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1)   # 32x32 -> 64x64
        self.deconv4 = nn.ConvTranspose2d(16, 8, kernel_size=4, stride=2, padding=1)    # 64x64 -> 128x128
        self.deconv5 = nn.ConvTranspose2d(8, 1, kernel_size=4, stride=2, padding=1)     # 128x128 -> 256x256

    def forward(self, x):
        # Fully connected layer to create an initial feature map
        x = self.fc(x)
        x = F.relu(x)
        x = x.view(-1, 128, 8, 8)  # Reshape to (batch_size, channels, height, width)

        # Upsample using transposed convolutions
        x = F.relu(self.deconv1(x))  # 16x16
        x = F.relu(self.deconv2(x))  # 32x32
        x = F.relu(self.deconv3(x))  # 64x64
        x = F.relu(self.deconv4(x))  # 128x128
        x = self.deconv5(x) 
        
        return x


In [252]:
from torch.utils.data import Dataset

class EmbeddingRasterDataset(Dataset):
    """
        Initializes the dataset with inputs (embeddings) and targets (rasters).

        Args:
            embeddings (numpy.ndarray or torch.Tensor): Input embeddings, shape (num_samples, 1024).
            rasters (numpy.ndarray or torch.Tensor): Target rasters, shape (num_samples, 256, 256).
    """
    def __init__(self, embeddings, rasters):
        self.embeddings = embeddings
        self.rasters = rasters

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        embedding = self.embeddings[idx]
        raster = self.rasters[idx]
        return torch.tensor(embedding, dtype=torch.float32), torch.tensor(raster, dtype=torch.float32)


In [ ]:
# DELETE?
index_to_drop = us_embs_with_polys_dropped_aug[us_embs_with_polys_dropped_aug['embeddings_array'].isna() == True].index[0]
us_embs_with_polys_dropped_aug = us_embs_with_polys_dropped_aug.drop(index = index_to_drop)

resampled_us_dropped = resampled_us[:index_to_drop] + resampled_us[index_to_drop+1:]

print(len(resampled_us_dropped))
print(len(us_embs_with_polys_dropped_aug))


In [ ]:
# 16
#resampled_us_256 = TrainingResample(filtered_results)

index_to_drop = us_embs_with_polys_dropped_aug[us_embs_with_polys_dropped_aug['embeddings_array'].isna() == True].index[0]
us_embs_with_polys_dropped_aug = us_embs_with_polys_dropped_aug.drop(index = index_to_drop)

resampled_us_dropped_256 = resampled_us_256[:index_to_drop] + resampled_us_256[index_to_drop+1:]

print(len(resampled_us_dropped_256))
print(len(us_embs_with_polys_dropped_aug))


In [ ]:
# 16 Clean training data and convert list of arrays to array
resampled_us16_np = resampled_us16
for i in range(len(resampled_us16)):
    resampled_us16_np[i] = np.array(resampled_us16[i])

resampled_us16_np = np.where(np.isnan(resampled_us16_np), 0, resampled_us16_np) 
resampled_us16_np = np.round_(resampled_us16_np)
resampled_us16_np = np.int_(resampled_us16_np)

In [579]:
# 17 Clean training data and convert list of arrays to array
resampled_us17_np = resampled_us17
for i in range(len(resampled_us17)):
    resampled_us17_np[i] = np.array(resampled_us17[i])

resampled_us17_np = np.where(np.isnan(resampled_us17_np), 0, resampled_us17_np) 
resampled_us17_np = np.round_(resampled_us17_np)
resampled_us17_np = np.int_(resampled_us17_np)

In [824]:
#16 Prep training data

us_embs_with_polys_dropped16 = us_embs_with_polys_dropped16.reset_index(drop=True)

index = sorted(random.sample(range(0, len(resampled_us16_np)), 25))

train_embs_with_polys16 = us_embs_with_polys_dropped16.drop(index = index)
train_embs_array16 = np.vstack(train_embs_with_polys16['embeddings'])
train_raster_array16 = np.stack([item for idx, item in enumerate(resampled_us16_np) if idx not in index])

test_embs_with_polys16 = us_embs_with_polys_dropped16.iloc[index]
test_embs_array16 = np.vstack(us_embs_with_polys_dropped16['embeddings'][index])
test_raster_array16 = np.stack([item for idx, item in enumerate(resampled_us16_np) if idx in index])

#train_raster_array_16 = np.where(np.isnan(train_raster_array), 0, train_raster_array) 
#train_raster_array_16 = np.round_(train_raster_array_filled16)
#train_raster_array_16 = np.int_(train_raster_array_filled_r16)
#test_raster_array16 = np.round_(test_raster_array16)
#test_raster_array16 = np.int_(test_raster_array16)

In [596]:
#17 Prep training data
us_embs_with_polys_dropped17 = us_embs_with_polys_dropped17.reset_index(drop=True)

index = sorted(random.sample(range(0, len(us_embs_with_polys_dropped17)), 25))

train_embs_with_polys17 = us_embs_with_polys_dropped17.drop(index = index)
train_embs_array17 = np.vstack(train_embs_with_polys17['embeddings'])
train_raster_array17 = np.stack([item for idx, item in enumerate(resampled_us17_np) if idx not in index])

test_embs_with_polys17 = us_embs_with_polys_dropped17.iloc[index]
test_embs_array17 = np.vstack(us_embs_with_polys_dropped17['embeddings'][index])
test_raster_array17 = np.stack([item for idx, item in enumerate(resampled_us17_np) if idx in index])

In [ ]:
# Control for error
#test_raster_array16 = np.where(test_raster_array16 == -2147483648, 0, test_raster_array16) 

print(np.unique(train_raster_array16, return_counts = True))
print(np.unique(test_raster_array16, return_counts = True))
print(np.unique(train_raster_array17, return_counts = True))
print(np.unique(test_raster_array17, return_counts = True))

In [ ]:
train_embs_array1617 = np.concatenate((train_embs_array16, train_embs_array17), axis = 0)
train_raster_array1617 = np.concatenate((train_raster_array16, train_raster_array17), axis = 0)

test_raster_array1617 = np.concatenate((test_raster_array16, test_raster_array17), axis = 0)
test_embs_array1617 = np.concatenate((test_embs_array16, test_embs_array17), axis = 0)

train_embs_with_polys1617 = pd.concat([train_embs_with_polys16, train_embs_with_polys17])
test_embs_with_polys1617 = pd.concat([test_embs_with_polys16, test_embs_with_polys17])

print(train_embs_array1617.shape)
print(test_raster_array1617.shape)
print(len(train_embs_with_polys1617))

In [ ]:
cnn_train_data = EmbeddingRasterDataset(train_embs_array1617, train_raster_array1617)

print(cnn_train_data.embeddings.shape)
print(cnn_train_data.rasters.shape)
print(test_raster_array.shape)

In [ ]:
# Drop out samples for 16 and 17 for ablation study

# Indexing from training embeddings, holding test embeddings constant
index = sorted(random.sample(range(0, len(train_embs_array1617)), 275))
train_embs_array1617_dropped = train_embs_with_polys1617.reset_index(drop = True)

train_embs_with_polys1617_dropped = train_embs_array1617_dropped.drop(index = index)
train_embs_array1617_dropped = train_embs_array1617[index]
train_raster_array1617_dropped = train_raster_array1617[index]

cnn_train_data = EmbeddingRasterDataset(train_embs_array1617_dropped, train_raster_array1617_dropped)

print(cnn_train_data.embeddings.shape)
print(cnn_train_data.rasters.shape)
print(test_raster_array1617.shape)

In [ ]:
# Filter out rasters with less than 65 positive pixels

valid_indices = []

# Iterate through each 256x256 slice in 'array'
for i in range(train_raster_array1617.shape[0]):
    if np.sum(train_raster_array1617[i]) >= 65:
        valid_indices.append(i)

# Convert valid_indices to numpy array (optional)
valid_indices = np.array(valid_indices)

# Filter 'array' and 'train_embs_array' using valid_indices
filtered_raster_array = train_raster_array1617[valid_indices]
filtered_train_embs_array = train_embs_array1617[valid_indices]
filtered_train_embs_with_polys = train_embs_with_polys1617.iloc[valid_indices]

print("Shape of filtered array:", filtered_raster_array.shape)
print("Shape of filtered train_embs_array:", filtered_train_embs_array.shape)

cnn_train_data = EmbeddingRasterDataset(filtered_train_embs_array, filtered_raster_array)


print(cnn_train_data.embeddings.shape)
print(cnn_train_data.rasters.shape)
print(test_raster_array.shape)


In [ ]:
from torch.utils.data import DataLoader
import torch.optim as optim
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assuming embeddings and rasters are numpy arrays of shape (num_samples, 1024) and (num_samples, 256, 256)
dataloader = DataLoader(cnn_train_data, batch_size=10, shuffle=True)

# Initialize model, loss, and optimizer
model = EmbeddingToRasterCNN().to(device)
num_positives = cnn_train_data.rasters.sum()
num_negatives = cnn_train_data.rasters.size - num_positives  # Total pixels - positives
#class_weight = min(num_negatives / num_positives, 100)  # Cap the weight
class_weight = (num_negatives / num_positives) / 2
print(class_weight)
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([class_weight]).to(device))
optimizer = optim.Adam(model.parameters(), lr=0.001)

def init_weights(m):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')  # He initialization
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)  # Initialize biases to zero

model.apply(init_weights)


# Debugging
torch.autograd.set_detect_anomaly(False)

def check_for_nans(module, input, output):
    print(f"Checking {module}")
    print(f"Output has NaNs? {torch.isnan(output).any()}")

hooks = []
for layer in model.modules():
    hook = layer.register_forward_hook(check_for_nans)
    hooks.append(hook)  # Keep track of the hooks
# Remove all hooks
for hook in hooks:
    hook.remove()

losses = []
# Training loop
epochs = 30
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_idx, (embeddings_batch, rasters_batch) in enumerate(dataloader):
        optimizer.zero_grad()

        #print(torch.isnan(rasters_batch).any())  # Check if rasters have NaN
        #print(torch.isnan(embeddings_batch).any())

        # Move data to device
        embeddings_batch = embeddings_batch.to(device)
        rasters_batch = rasters_batch.to(device)

        # Forward pass
        outputs = model(embeddings_batch)

        # Compute loss
        outputs = torch.clamp(outputs, min=-3, max=3)  # Prevent extreme values
        loss = criterion(outputs, rasters_batch.unsqueeze(1))  # Add channel dimension to targets
        total_loss += loss.item()

        # Backward pass
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Update weights
        optimizer.step()

        # Logging batch loss
        #print(f"Epoch {epoch + 1}, Batch {batch_idx + 1}/{len(dataloader)}, Loss: {loss.item()}")

        #for name, param in model.named_parameters():
        #    if param.requires_grad:
        #        print(f"{name}: max={param.data.max()}, min={param.data.min()}, grad_max={param.grad.max()}, grad_min={param.grad.min()}")


    # Logging epoch loss
    print(f"Epoch {epoch + 1}/{epochs}, Avg Loss: {total_loss / len(dataloader)}")
    losses.append(total_loss/len(dataloader))


import matplotlib.pyplot as plt
epochs_x = np.linspace(1, epochs, epochs)
plt.plot(epochs_x, losses)
plt.show()

#### Evaluation

In [ ]:
# Set the model to evaluation mode for one sample
model.eval()
criterion = nn.BCEWithLogitsLoss(reduction = 'mean')
prediction_losses = []

for i in range(10,15):
    sample_embedding = torch.tensor(test_embs_array1617[i], dtype=torch.float32).unsqueeze(0)  # Add batch dimension
    ground_truth = test_raster_array1617[i].flatten()

    # Run inference (no gradients needed for inference)
    with torch.no_grad():
        predicted_raster = model(sample_embedding)  # Shape of predicted_raster will be [1, 1, 256, 256]
        predictions = torch.sigmoid(predicted_raster) > 0.5
        actuals = torch.tensor(test_raster_array1617[i].reshape(1, 1, 256, 256))
        loss = criterion(predicted_raster, actuals.float())
        print(loss)
        prediction_losses.append(loss)
        print(test_embs_with_polys1617.to_crs(epsg=4326)['geometry'].iloc[i].centroid)


    rasterio.plot.show(np.array(predictions))
    rasterio.plot.show(test_raster_array1617[i])

In [ ]:
prediction_losses_df = pd.DataFrame(np.array(prediction_losses), columns = ['losses'])
#plt.hist(prediction_losses_np)

mean_value = prediction_losses_df.mean()           # Mean
median_value = prediction_losses_df.median()      # Median
percentiles = prediction_losses_df.quantile([0.1, 0.5, 0.9])

print("Mean:", mean_value)
print("Median:", median_value)
print(percentiles)


smaller_losses = prediction_losses_df[prediction_losses_df < 0.13]
plt.hist(smaller_losses)
plt.title('90th Percentile Prediction Losses')
len(smaller_losses)

In [ ]:
bigger_losses = prediction_losses_np[prediction_losses_np > 0.13]
print(bigger_losses)

In [ ]:
# Set the model to evaluation mode for train set
model.eval()
criterion = nn.BCEWithLogitsLoss(reduction = 'mean')
train_prediction_losses = []
train_all_ground_truths = []
train_all_predictions = []

for i in range(0,50):
    sample_embedding = torch.tensor(train_embs_array1617[i], dtype=torch.float32).unsqueeze(0)  # Add batch dimension

    # Run inference (no gradients needed for inference)
    with torch.no_grad():
        predicted_raster = model(sample_embedding)  # Shape of predicted_raster will be [1, 1, 256, 256]
        predictions = torch.sigmoid(predicted_raster) > 0.5
        actuals = torch.tensor(train_raster_array1617[i].reshape(1, 1, 256, 256))
        ground_truth = actuals.squeeze(1).numpy()
        loss = criterion(predicted_raster, actuals.float())
        print(loss)
        prediction_losses.append(loss)
        predictions_np = predictions.squeeze(1).numpy()

    train_all_predictions.extend(predictions_np)
    train_all_ground_truths.extend(ground_truth)

    #rasterio.plot.show(np.array(predictions))
    #rasterio.plot.show(test_raster_array1617[i])

    #print(train_embs_with_polys1617.to_crs(epsg=4326)['geometry'].iloc[i].centroid)

# Now `predicted_raster` is your predicted raster output
#print(predicted_raster.shape)

train_all_predictions = np.array(train_all_predictions)
train_all_ground_truths = np.array(train_all_ground_truths)

In [ ]:
# Training Predictions Metrics
from sklearn.metrics import precision_score, recall_score, f1_score
flattened_ground_truths = train_all_ground_truths.flatten().astype(int)  # Convert to 1D and numpy
flattened_predictions = train_all_predictions.flatten().astype(int)  # Convert to 1D and numpy

precision = precision_score(flattened_ground_truths, flattened_predictions, average='binary')
recall = recall_score(flattened_ground_truths, flattened_predictions, average='binary')
f1 = f1_score(flattened_ground_truths, flattened_predictions, average='binary')

print(f'precision on training: {precision}')
print(f'recall on training: {recall}')
print(f'f1 on training: {f1}')

In [ ]:
# Set the model to evaluation mode for test set
model.eval()
criterion = nn.BCEWithLogitsLoss(reduction = 'mean')
prediction_losses = []
all_ground_truths = []
all_predictions = []

for i in range(0,50):
    sample_embedding = torch.tensor(test_embs_array1617[i], dtype=torch.float32).unsqueeze(0)  # Add batch dimension

    # Run inference (no gradients needed for inference)
    with torch.no_grad():
        predicted_raster = model(sample_embedding)  # Shape of predicted_raster will be [1, 1, 256, 256]
        predictions = torch.sigmoid(predicted_raster) > 0.5
        actuals = torch.tensor(test_raster_array1617[i].reshape(1, 1, 256, 256))
        ground_truth = actuals.squeeze(1).numpy()
        loss = criterion(predicted_raster, actuals.float())
        print(loss)
        prediction_losses.append(loss)
        predictions_np = predictions.squeeze(1).numpy()

    all_predictions.extend(predictions_np)
    all_ground_truths.extend(ground_truth)

    rasterio.plot.show(np.array(predictions))
    rasterio.plot.show(test_raster_array1617[i])

    print(test_embs_with_polys1617.to_crs(epsg=4326)['geometry'].iloc[i].centroid)

# Now `predicted_raster` is your predicted raster output
#print(predicted_raster.shape)

all_predictions = np.array(all_predictions)
all_ground_truths = np.array(all_ground_truths)

print(all_ground_truths.flatten().shape)
print(all_predictions.flatten().shape)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
flattened_ground_truths = all_ground_truths.flatten().astype(int)  # Convert to 1D and numpy
flattened_predictions = all_predictions.flatten().astype(int)  # Convert to 1D and numpy

precision = precision_score(flattened_ground_truths, flattened_predictions, average='binary')
recall = recall_score(flattened_ground_truths, flattened_predictions, average='binary')
f1 = f1_score(flattened_ground_truths, flattened_predictions, average='binary')

print(f'precision on test: {precision}')
print(f'recall on test: {recall}')
print(f'f1 on test: {f1}')

In [ ]:
# Set the model to evaluation mode
model.eval()

for i in range(0,10):
    sample_embedding = torch.tensor(train_embs_with_polys['embeddings'].iloc[i], dtype=torch.float32).unsqueeze(0)  # Add batch dimension

    # Run inference (no gradients needed for inference)
    with torch.no_grad():
        predicted_raster = model(sample_embedding)  # Shape of predicted_raster will be [1, 1, 256, 256]

    rasterio.plot.show(np.array(predicted_raster))
    rasterio.plot.show(train_raster_array[i])

# Now `predicted_raster` is your predicted raster output
#print(predicted_raster.shape)

#rasterio.plot.show(np.array(predicted_raster))

In [ ]:
stack1 = RetrievePixels(q1stack)
stack2 = RetrievePixels(q2stack)
stack3 = RetrievePixels(q3stack)
stack4 = RetrievePixels(q4stack)

In [ ]:
# Get median of values across time for different bands for each season
import xarray as xr

q1_composite = stack1.groupby('band').median(dim = 'time')
q2_composite = stack2.groupby('band').median(dim = 'time')
q3_composite = stack3.groupby('band').median(dim = 'time')
q4_composite = stack4.groupby('band').median(dim = 'time')

# Calculate NDWI and create water mask 
def CalculateNDWI(image):
    green = image.sel(band="green")
    nir = image.sel(band="nir")

    ndwi = (green - nir) / (green + nir)
    return ndwi

ndwi_q1 = CalculateNDWI(q1_composite)
ndwi_q2 = CalculateNDWI(q2_composite)
ndwi_q3 = CalculateNDWI(q3_composite)
ndwi_q4 = CalculateNDWI(q4_composite)

ndwi_ds = [ndwi_q1, ndwi_q2, ndwi_q3, ndwi_q4]

for i in range(len(ndwi_ds)):
    if 's2:processing_baseline' in ndwi_ds[i].coords:
        ndwi_ds[i] = ndwi_ds[i].reset_coords('s2:processing_baseline', drop=True)
        
annual_median = xr.concat(ndwi_ds, dim="time").median(dim='time')


In [ ]:
import rioxarray as rxr

# Get water mask, get intersection with pit lakes as positive
# Reproject  
water_mask = annual_median.rio.write_crs('epsg:32615', inplace=True)
water_mask = water_mask.rio.reproject('epsg:4326')
water_mask = xr.where(water_mask > 0.1, 1, 0)

Label water

In [ ]:
# Re-label bounding boxes with water mask
# If bbox indicates pit lake & NDWI indicates water, label as positive. Else, negative

# If pixel is labeled 1, check whether raster is 1 or 0
# Resample water mask to match polygons
water_mask_resampled = resample_raster(
        src2, 
        target_transform=transform1,
        target_shape=raster1.shape,
        resampling=Resampling.nearest
    )

# Get the corresponding pixel values from the second raster
mask = bboxs_raster == 1
bboxs_raster[mask] = water_mask_resampled[mask]

rasterio.plot.show(bboxs_raster)

#rasters.append(bboxs_raster)
#embs_with_polys()

# Add the statistics to the vector data
#testing = mn_qa_bboxs.join(pd.DataFrame(water_mask_chips))
#testing.columns
#mn_data = mn_data.join(pd.DataFrame(water_mask_chips))



In [ ]:
# Get chips with pit lakes and regular bodies of water in MN from QA
mn_data = LabelEmbeddings(mine_embeddings_mn, lake_embeddings_mn, mn_qa_polys, dnr_water_cropped)
print(f"Total samples: {len(mn_data)}") 


# Get chips for features identified as not lakes from QA
mn_nonlakes_chips = LabelEmbeddings(dummy, qa_non_mine_embeddings_mn, gpd.GeoDataFrame(), mn_qa_nonlakes)
print(f"Total samples: {len(mn_nonlakes_chips)}") 

mn_data = pd.concat([mn_data,mn_nonlakes_chips], ignore_index=True) # Add lakes to MN embeddings

### PCA and tSNE

In [63]:
# Define function to get principle components and plot with data for each embedding

def EmbeddingsPCA(data, column = 'embeddings'):
    """
    
    Parameters: 
    data (GeoDataFrame):
    column (str): column of data with variable

    """
    # Convert embeddings to array and do PCA
    X =  np.vstack(data[column].values)
    pca = PCA(n_components=1)
    pca.fit(X)

    # Check on variance explained by given # of dimensions of PCA
    print("Relative variance in principal components:", pca.explained_variance_ratio_)

    # Apply PCA to get first two dimensions
    data['pca1'] = pca.transform(X)[:, 0]
    #data['pca2'] = pca.transform(X)[:, 1]

    return data

In [65]:
def PCAVisualization(data, column, label1, label2):
    # Visualize the first two principal components with category color-coding
    fig, ax = plt.subplots()

    # Plot for category 1
    ax.scatter(data[data[column] == True]['pca1'], 
                data[data[column] == True]['pca2'], 
                color='blue', label=label1, alpha=0.5)

    # Plot for category 2
    ax.scatter(data[data[column] == False]['pca1'], 
                data[data[column] == False]['pca2'], 
                color='grey', label=label2, alpha=0.05)
    
    """ EDIT SO CAN SEE WHAT THE LITTLE OVERLAP IS  plt.scatter(data[data['wb_'] == new_condition]['pca1'], 
            data[data[new_class_column] == new_condition]['pca2'], 
            color='red', label=label3, alpha=0.5, marker='^') """

    plt.xlabel('PCA 1')
    plt.ylabel('PCA 2')
    ax.legend()
    plt.title('PCA of NAIP Embeddings')

    return ax

In [ ]:
mn_pca = EmbeddingsPCA(mn_data)
PCAVisualization(mn_pca, column = 'mine', label1 = 'Pit lakes', label2 = 'Non-pit lake water bodies')

In [ ]:
az_pca = EmbeddingsPCA(az_data)
ax = PCAVisualization(az_pca, 'mine','Pit lakes', 'No pit lakes present')

end_idx = len(az_pca)-1
start_idx = end_idx-len(az_lakes_chips)

ax.scatter(az_pca[start_idx:end_idx]['pca1'], 
                az_pca[start_idx:end_idx]['pca2'],
                color='green', label = 'lake', alpha = 0.1)

ax.legend()
plt.show()

In [ ]:
#t-SNE of positive/negative embeddings

labels = data['mine'].values

tsne_data = np.vstack(data['embeddings'].values)

# Set up t-SNE
tsne = TSNE(n_components=2, perplexity=25, n_iter=250)
reduced_embeddings = tsne.fit_transform(tsne_data)

# Visualize the results
plt.figure(figsize=(8, 6))

custom_labels = {0: "Not Pit Lakes", 1: "Pit Lakes"}

for label in np.unique(labels):
    mask = labels == label
    plt.scatter(reduced_embeddings[mask, 1],
                reduced_embeddings[mask, 0],
                label=custom_labels.get(label, f"Label {label}"),
                alpha=0.7)

plt.title("t-SNE of Minnesota Embeddings")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.grid(True)
plt.legend()
plt.show()

### Build classification model to predict presence of pit lakes

In [909]:
# Mark positive samples 
import math

def MarkSamples(data, polygons, pct):
    """ Mark as positive if intersection is greater than given % of area.
    """
    # Get polygon of intersection for each chip
    positive_data = data[data['mine'] == 1]
    negative_data = data[data['mine'] == 0]
    positive_data['emb_idx'] = positive_data.index
    intersecting = gpd.overlay(positive_data, polygons, how = 'intersection') # Returns polygons of intersection
    intersecting = intersecting.dissolve(by='emb_idx') # Makes intersecting index match with data, adds areas of intersection
    intersecting['overlapping_area'] = intersecting.geometry.area

    # Merge into one dataframe 
    positive_data = positive_data.merge(intersecting['overlapping_area'], left_index = True, right_index = True, how='left')

    # Check intersection % and mark True/False
    for index, row in positive_data.iterrows():
        pct_overlap = row['overlapping_area'] / row.geometry.area
        positive_data.loc[index,'pct_pitlake'] = pct_overlap
        if pct_overlap >= pct:
            positive_data['mine'][index] = 1
        elif math.isnan(row['overlapping_area']):
            positive_data['mine'][index] = 0
        else: 
            positive_data['mine'][index]  = 0

    data = pd.concat([positive_data, negative_data])

    return data

In [ ]:
# Larger US

# Get 16 and 17
usdata1617 = pd.concat([data16R, data16S, data16T, data17R, data17S, data17T])
usdata1617 = usdata1617.drop_duplicates('geometry')

print(len(us_qa_pits_16)+len(us_qa_pits_17))
us_qa_pits_1617 = pd.concat([us_qa_pits_16, us_qa_pits_17]).drop_duplicates('geometry')
print(len(us_qa_pits_1617))

# Check if intersection of each chip with a pit lake polygon is > 0.03%
usdata1617marked = MarkSamples(usdata1617, us_qa_pits_1617, 0.0003)
print((usdata1617marked['mine']==1).sum())
print(len(usdata1617marked))

# Split into train and test
usdata1617marked = usdata1617marked.reset_index(drop=True)
us1617_train_embeddings = np.vstack(usdata1617marked['embeddings'].values)
us1617_labels = np.array(usdata1617marked['mine'])
us1617_indices = usdata1617marked.index

us1617_X_train, us1617_X_test, us1617_y_train, us1617_y_test, us1617_train_idx, us1617_test_idx = sklearn.model_selection.train_test_split(us1617_train_embeddings, us1617_labels, us1617_indices, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)

print(np.shape(us1617_X_train))
print(np.shape(us1617_X_test))
print(np.shape(us1617_y_train))
print(np.shape(us1617_y_test))

In [ ]:
usdata1617marked['pct_pitlake'].fillna(0, inplace=True)
usdata1617marked['pct_pitlake']

In [ ]:
# Larger US REGRESSOR

# Split into train and test
usdata1617marked = usdata1617marked.reset_index(drop=True)
us1617_train_embeddings = np.vstack(usdata1617marked['embeddings'].values)
us1617_labels = np.array(usdata1617marked['pct_pitlake'])
us1617_indices = usdata1617marked.index

us1617_X_train, us1617_X_test, us1617_y_train, us1617_y_test, us1617_train_idx, us1617_test_idx = sklearn.model_selection.train_test_split(us1617_train_embeddings, us1617_labels, us1617_indices, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)

print(np.shape(us1617_X_train))
print(np.shape(us1617_X_test))
print(np.shape(us1617_y_train))
print(np.shape(us1617_y_test))

In [950]:
#usdata1617marked = usdata1617marked[usdata1617marked['mine'] == 1]

In [ ]:
usdata1617marked['pct_pitlake'].mean()

In [ ]:
plt.hist(usdata1617marked['pct_pitlake'])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Create and train the RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100, random_state=1)
regressor.fit(us1617_X_train, us1617_y_train)

# Make predictions
y_pred = regressor.predict(us1617_X_test)

# Evaluate the model
mse = mean_squared_error(us1617_y_test, y_pred)
print(f"Mean Squared Error: {mse}")
math.sqrt(mse)

In [ ]:
fig, ax = plt.subplots()

plt.xlim(0,0.08)
plt.ylim(0,0.08)
ax.scatter(regressor.predict(us1617_X_train), us1617_y_train, color = 'grey')
ax.scatter(y_pred, us1617_y_test, color = 'blue')
x = np.linspace(0,1)
y = np.linspace(0,1)
ax.plot(x, y, linestyle = ':')
ax.legend(['train', 'test'])
ax.set_xlabel('Predicted')
ax.set_ylabel('True')

plt.show()

In [ ]:
# Train neural network model for IN and MN
#from sklearn.utils.class_weight import compute_class_weight

#class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=mn_y_train)

# Convert the class weights to a dictionary
#class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

us1617_clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(16, 16, 16), max_iter = 1000, 
                    random_state=1)

us1617_clf.fit(us1617_X_train, us1617_y_train)

In [ ]:
# Run predictions on test set and get F1 scores and probabilities

us1617_predictions = us1617_clf.predict(us1617_X_test) 
print(f'Number of positive predictions: {us1617_predictions.sum()}')
print(f'Number of positive samples: {us1617_y_test.sum()}')
print(f'Number of test samples: {len(us1617_X_test)}')

f1 = sklearn.metrics.f1_score(us1617_y_test, us1617_predictions)
print(f'F1: {f1}')

in_mn_precision = sklearn.metrics.precision_score(us1617_y_test, us1617_predictions)
print(f'precision: {in_mn_precision}')

in_mn_recall = sklearn.metrics.recall_score(us1617_y_test, us1617_predictions)
print(f'recall: {in_mn_recall}')

us1617_probabilities = us1617_clf.predict_proba(us1617_X_test)
print(f'Classes for probability: {us1617_clf.classes_}')

us1617_probabilities[:,1] # Probability of pit lake presence

In [ ]:
# Write predictions and probabilities to shapefiles
in_mn_pits_geom = in_mn_data_marked_reset.loc[in_mn_test_idx].geometry.reset_index(drop = True)

in_mn_pits_pred = gpd.GeoDataFrame(in_mn_predictions, columns = ['mines'], geometry=in_mn_pits_geom, crs = 'EPSG:4326')
in_mn_pits_probs = gpd.GeoDataFrame(in_mn_probabilities[:,1], columns = ['mine_prob'], geometry=in_mn_pits_geom, crs = 'EPSG:4326')

in_mn_pits_probs.to_file('north_pits_probs_0112')

#### Minnesota & Indiana

In [ ]:
# Get MN, IN, KY data
in_mn_data = pd.concat([in_data, mn_data])
north_data = pd.concat([in_mn_data, ky_data])
in_mn_pits = pd.concat([mn_aggregated_pits, in_qa_pits])
north_pits = pd.concat([in_mn_pits, ky_qa_pits])

# Check if intersection of each chip with a pit lake polygon is > 0.5%
in_mn_data_marked = MarkSamples(north_data, north_pits, 0.006)
print((in_mn_data_marked['mine']==1).sum())
print(len(in_mn_data_marked))

# Split into train and test
in_mn_data_marked_reset = in_mn_data_marked.reset_index(drop=True)
in_mn_train_embeddings = np.vstack(in_mn_data_marked_reset['embeddings'].values)
in_mn_labels = np.array(in_mn_data_marked_reset['mine'])
in_mn_indices = in_mn_data_marked_reset.index

in_mn_X_train, in_mn_X_test, in_mn_y_train, in_mn_y_test, in_mn_train_idx, in_mn_test_idx = sklearn.model_selection.train_test_split(in_mn_train_embeddings, in_mn_labels, in_mn_indices, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)

print(np.shape(in_mn_X_train))
print(np.shape(in_mn_X_test))
print(np.shape(in_mn_y_train))
print(np.shape(in_mn_y_test))

In [ ]:
# Train neural network model for IN and MN
from sklearn.utils.class_weight import compute_class_weight

#class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=mn_y_train)

# Convert the class weights to a dictionary
#class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

in_mn_clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(16, 16, 16), max_iter = 1000, 
                    random_state=1)

in_mn_clf.fit(in_mn_X_train, in_mn_y_train)

In [ ]:
# Run predictions on test set and get F1 scores and probabilities

in_mn_predictions = in_mn_clf.predict(in_mn_X_test) 
print(f'Number of positive predictions: {in_mn_predictions.sum()}')
print(f'Number of positive samples: {in_mn_y_test.sum()}')
print(f'Number of test samples: {len(in_mn_X_test)}')

f1 = sklearn.metrics.f1_score(in_mn_y_test, in_mn_predictions)
print(f'F1: {f1}')

in_mn_precision = sklearn.metrics.precision_score(in_mn_y_test, in_mn_predictions)
print(f'precision: {in_mn_precision}')

in_mn_recall = sklearn.metrics.recall_score(in_mn_y_test, in_mn_predictions)
print(f'recall: {in_mn_recall}')

in_mn_probabilities = in_mn_clf.predict_proba(in_mn_X_test)
print(f'Classes for probability: {in_mn_clf.classes_}')

in_mn_probabilities[:,1] # Probability of pit lake presence

In [1161]:
# Write predictions and probabilities to shapefiles
in_mn_pits_geom = in_mn_data_marked_reset.loc[in_mn_test_idx].geometry.reset_index(drop = True)

in_mn_pits_pred = gpd.GeoDataFrame(in_mn_predictions, columns = ['mines'], geometry=in_mn_pits_geom, crs = 'EPSG:4326')
in_mn_pits_probs = gpd.GeoDataFrame(in_mn_probabilities[:,1], columns = ['mine_prob'], geometry=in_mn_pits_geom, crs = 'EPSG:4326')

in_mn_pits_probs.to_file('north_pits_probs_0112')

In [ ]:
import joblib
joblib.dump(in_mn_clf, 'data/NorthPitLakeClf0112.pkl')

#### Minnesota

In [ ]:
# Check if intersection of each chip with a pit lake polygon is > 0.5%
mn_data_marked = MarkSamples(mn_data, mn_aggregated_pits, 0.005)
print((mn_data_marked['mine']==1).sum())
print(len(mn_data_marked))


In [ ]:
mn_data_marked.to_file('mn_coverage_check_112')

In [ ]:
# Split into train and test for MN

mn_data_marked_reset = mn_data_marked.reset_index(drop=True)
mn_train_embeddings = np.vstack(mn_data_marked_reset['embeddings'].values)
mn_labels = np.array(mn_data_marked_reset['mine'])
mn_indices = mn_data_marked_reset.index

mn_X_train, mn_X_test, mn_y_train, mn_y_test, mn_train_idx, mn_test_idx = sklearn.model_selection.train_test_split(mn_train_embeddings, mn_labels, mn_indices, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)

print(np.shape(mn_X_train))
print(np.shape(mn_X_test))
print(np.shape(mn_y_train))
print(np.shape(mn_y_test))

In [ ]:
# Train neural network model
from sklearn.utils.class_weight import compute_class_weight

#class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=mn_y_train)

# Convert the class weights to a dictionary
#class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

mn_clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(32, 32, 32), max_iter = 1000, 
                    random_state=1)

mn_clf.fit(mn_X_train, mn_y_train)

In [ ]:
# Run predictions on test set and get F1 scores and probabilities

mn_predictions = mn_clf.predict(mn_X_test) 
print(f'Number of positive predictions: {mn_predictions.sum()}')
print(f'Number of positive samples: {mn_y_test.sum()}')
print(f'Number of test samples: {len(mn_X_test)}')

f1 = sklearn.metrics.f1_score(mn_y_test, mn_predictions)
print(f'F1: {f1}')

mn_precision = sklearn.metrics.precision_score(mn_y_test, mn_predictions)
print(f'precision: {mn_precision}')

mn_probabilities = mn_clf.predict_proba(mn_X_test)
print(f'Classes for probability: {mn_clf.classes_}')

mn_probabilities[:,1] # Probability of pit lake presence

In [ ]:
import joblib
joblib.dump(mn_clf, 'data/MNPitLakeClf0109.pkl')

In [966]:
# Write predictions and probabilities to shapefiles
mn_pits_geom = mn_data_marked_reset.loc[mn_test_idx].geometry.reset_index(drop = True)

mn_pits_pred = gpd.GeoDataFrame(mn_predictions, columns = ['mines'], geometry=mn_pits_geom, crs = 'EPSG:4326')
mn_pits_probs = gpd.GeoDataFrame(mn_probabilities[:,1], columns = ['mine_prob'], geometry=mn_pits_geom, crs = 'EPSG:4326')

mn_pits_probs.to_file('mn_pits_probs_0112')

In [ ]:
mn_train_predictions = mn_clf.predict(mn_X_train)

mn_f1_train = sklearn.metrics.f1_score(mn_y_train, mn_train_predictions)
print(f'F1 on training data: {mn_f1_train}')

mn_train_accuracy = sklearn.metrics.accuracy_score(mn_y_train, mn_train_predictions)
mn_test_accuracy = sklearn.metrics.accuracy_score(mn_y_test, mn_predictions)
print(f'Accuracy on training data: {mn_train_accuracy}')
print(f'Accuracy on test data: {mn_test_accuracy}')

# Plot histogram of probability predictions on test set
test_probs = pd.DataFrame(mn_probabilities)
bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
test_probs[1].hist(bins = bins)

In [ ]:
# Get embeddings for Crosby, MN
crosby_pitlakes = gpd.read_file('data/crosby_mn.geojson') # all Crosby

crosby_embeddings, dummy = GetEmbeddingsFromIntersection1_5(folder_path = 'data/MN_v1_5/', polygons = crosby_pitlakes, random_images = False)

print("Number of images:")
print(len(crosby_embeddings))

# Get Crosby test chips
crosby_embeddings = pd.DataFrame(crosby_embeddings)
crosby_embs_avg = MonthlyComposite(crosby_embeddings)
print(len(crosby_embs_avg))
crosby_test_chips = LabelEmbeddings1_5(crosby_embs_avg, dummy, crosby_pitlakes) #  getting all Crosby, not just pit lakes

print("Total samples:") 
print(len(crosby_test_chips))

crosby_test = np.vstack(crosby_test_chips['embeddings'].values) 

crosby_test_preds = gpd.GeoDataFrame(mn_clf.predict(crosby_test), columns = ['mine'], geometry = crosby_test_chips['geometry'])
crosby_test_probs = gpd.GeoDataFrame(mn_clf.predict_proba(crosby_test)[:,1], columns = ['mine_prob'], geometry=crosby_test_chips['geometry'], crs = 'EPSG:4326')

crosby_test_probs.plot(column = 'mine_prob', legend = True)

# Write Crosby predictions to shapefile
crosby_test_probs.to_file('crosby_testing_0112')

In [ ]:
# Test with IN and MN combined model

crosby_test_preds = gpd.GeoDataFrame(in_mn_clf.predict(crosby_test), columns = ['mine'], geometry = crosby_test_chips['geometry'])
crosby_test_probs = gpd.GeoDataFrame(in_mn_clf.predict_proba(crosby_test)[:,1], columns = ['mine_prob'], geometry=crosby_test_chips['geometry'], crs = 'EPSG:4326')

crosby_test_probs.plot(column = 'mine_prob', legend = True)

# Write Crosby predictions to shapefile
#crosby_test_probs.to_file('crosby_testing_0112_no')

In [ ]:
(crosby_test_preds['mine']==1).sum()

dnr_pit_lakes_crosby = mn_water_features.loc[mn_water_features['wb_class'].isin(mine_classes)].to_crs(epsg=4326)

crosby_pits = gpd.sjoin(dnr_pit_lakes_crosby, crosby_pitlakes, how='inner', predicate = 'intersects')

crosby_marked = MarkSamples(crosby_test_chips, crosby_pits, 0.002)

print(crosby_marked['mine'].sum())
len(crosby_test_probs)

In [ ]:
# Get embeddings for MI
mi_qa = gpd.read_file('data/mi_testing.geojson')

mi_qa_embeddings, dummy = GetEmbeddingsFromIntersection(folder_path = 'data/60cm_mi/rgbir_cog/', polygons = mi_qa, random_images = False)

print("Number of images:")
print(len(mi_qa_embeddings))

# Get MI test chips
mi_test_chips = LabelEmbeddings(mi_qa_embeddings, dummy, mi_qa)

print("Total samples:") 
print(len(mi_test_chips))

mi_test = np.vstack(mi_test_chips['embeddings'].values) 

mi_test_preds = gpd.GeoDataFrame(mn_clf.predict(mi_test), columns = ['mine'], geometry = mi_test_chips['geometry'])
mi_test_probs = gpd.GeoDataFrame(mn_clf.predict_proba(mi_test)[:,1], columns = ['mine_prob'], geometry=mi_test_chips['geometry'], crs = 'EPSG:4326')

# Write Crosby predictions to shapefile
mi_test_probs.to_file('mi_testing_1218')

#### Test on MI lakes vs. pit lakes

In [ ]:
# Get lake data - filter out pit lakes

mi_lakes = gpd.read_file('data/mi_Lake_Polygons.geojson').to_crs(epsg = 4326)
states = gpd.read_file('data/state_boundaries')
mi = states[states['STUSPS']=='MI']
mi = mi.to_crs(epsg = 4326).geometry.unary_union 
mi_qa = US_QA[US_QA.geometry.intersects(mi)]

mi_qa_pits = mi_qa[mi_qa['category'].isin(['a','q'])] # QAed pit lakes and questionable
mi_lakes_polys = gpd.overlay(mi_lakes, mi_qa_pits, how = 'difference')
print(len(mi_lakes_polys))

# Get pit lake polygon intersections - keep polygons within pit lakes; otherwise keep bbox
mi_qa_polys = gpd.overlay(mi_qa_pits, mi_lakes, how = 'intersection')

bboxs_with_polygon = gpd.sjoin(mi_qa_pits, mi_lakes, how="left", predicate="intersects")
print(len(bboxs_with_polygon))

bboxs_without_polygon = bboxs_with_polygon[bboxs_with_polygon.index_right.isna()]
print(len(bboxs_without_polygon))

mi_qa_polys = pd.concat([mi_qa_polys, bboxs_without_polygon])
mi_qa_polys.plot()

In [ ]:
# Get intersecting embeddings, label these
mi_embeddings, dummy = GetEmbeddingsFromIntersection(folder_path = 'data/60cm_mi/rgbir_cog/', polygons = mi_qa_polys, random_images = False)

print("Number of mine images:")
print(len(mi_embeddings))

# Crop
random_idx = random.sample(range(1, len(mi_lakes_polys)), 500)
mi_lakes_cropped = mi_lakes_polys.iloc[random_idx]

In [ ]:
# Get lake intersecting embeddings
mi_water_embeddings, dummy = GetEmbeddingsFromIntersection(folder_path = 'data/60cm_mi/rgbir_cog/', polygons = mi_lakes_cropped, random_images = False)
print(f"Number of water images: {len(mi_embeddings)}")

# Get MI test chips
mi_test_chips = LabelEmbeddings(mi_embeddings, mi_water_embeddings, mi_qa_polys, mi_lakes_cropped)

print("Total samples:") 
print(len(mi_test_chips))


# Predict on these
mi_test = np.vstack(mi_test_chips['embeddings'].values) 
mi_test_labels = mi_test_chips['mine']

mi_test_preds = gpd.GeoDataFrame(mn_clf.predict(mi_test), columns = ['mine'], geometry = mi_test_chips['geometry'])
mi_test_probs = gpd.GeoDataFrame(mn_clf.predict_proba(mi_test)[:,1], columns = ['mine_prob'], geometry=mi_test_chips['geometry'], crs = 'EPSG:4326')


In [ ]:
mi_test_preds_list = mn_clf.predict(mi_test)

mi_precision = sklearn.metrics.precision_score(mi_test_labels, mi_test_preds_list)
print(f'precision: {mi_precision}')

mi_f1 = sklearn.metrics.f1_score(mi_test_labels, mi_test_preds_list)
print(f'F1 on MI: {mi_f1}')

In [54]:
# Write MI predictions to shapefile
mi_test_probs.to_file('mi_testing_1219_2p')


#### Arizona

In [ ]:
# Mark as positive if intersection > 30% 
az_data_marked_reset = az_data.reset_index(drop=True)
az_data_marked = MarkSamples(az_data_marked_reset, az_qa_pits, 0.003)

print((az_data_marked['mine']==1).sum())

In [ ]:
# Split into train and test for AZ

az_train_embeddings =  np.vstack(az_data_marked['embeddings'].values)
az_labels = np.array(az_data_marked['mine'])
az_indices = az_data_marked.index

az_X_train, az_X_test, az_y_train, az_y_test, az_train_idx, az_test_idx = sklearn.model_selection.train_test_split(az_train_embeddings, az_labels, az_indices, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)

print(np.shape(az_X_train))
print(np.shape(az_X_test))
print(np.shape(az_y_train))
print(np.shape(az_y_test))

In [ ]:
# Train neural network model

az_clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(16, 16, 16), max_iter = 1000, random_state=1)

az_clf.fit(az_X_train, az_y_train)

In [ ]:
az_predictions = az_clf.predict(az_X_test) 
print(f'Number of positive predictions: {az_predictions.sum()}')
print(f'Number of test samples: {len(az_X_test)}')

az_f1 = sklearn.metrics.f1_score(az_y_test, az_predictions)
print(f'F1: {f1}')

probabilities = az_clf.predict_proba(az_X_test)
print(f'Classes for probability: {az_clf.classes_}')

probabilities[:,1] # Probability of pit lake presence

In [ ]:
az_train_predictions = az_clf.predict(az_X_train)

az_f1_train = sklearn.metrics.f1_score(az_y_train, az_train_predictions)
print(f'F1 on training data: {az_f1_train}')

train_accuracy = sklearn.metrics.accuracy_score(az_y_train, az_train_predictions)
test_accuracy = sklearn.metrics.accuracy_score(az_y_test, az_predictions)
print(f'Accuracy on training data: {train_accuracy}')
print(f'Accuracy on test data: {test_accuracy}')

# Plot histogram of probability predictions on test set
test_probs = pd.DataFrame(probabilities)
bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
test_probs[1].hist(bins = bins)

In [ ]:
# Write predictions and probabilities to shapefiles
az_pits_geom = az_data.geometry.loc[az_test_idx].reset_index(drop = True)
az_pits_pred = gpd.GeoDataFrame(az_predictions, columns = ['mines'], geometry=az_pits_geom, crs = 'EPSG:4326')
az_pits_probs = gpd.GeoDataFrame(probabilities[:,1], columns = ['mine_prob'], geometry=az_pits_geom, crs = 'EPSG:4326')

#az_pits_pred.to_file('az_pits_pred_1')
az_pits_probs.to_file('az_pits_probs_0108')

In [ ]:
# Get embeddings for Miami, AZ pitlakes
miami_pitlakes = gpd.read_file('data/miami_az_pits.geojson')

miami_embeddings, dummy = GetEmbeddingsFromIntersection(folder_path = 'data/60cm_az/rgbir_cog/', polygons = miami_pitlakes, random_images = False)

print("Number of images:")
print(len(miami_embeddings))

# Get test chips in AZ
miami_test_chips = LabelEmbeddings(miami_embeddings, dummy, miami_pitlakes)

print("Total samples:") 
print(len(miami_test_chips))

miami_test = np.vstack(miami_test_chips['embeddings'].values) 

miami_test_preds = gpd.GeoDataFrame(az_clf.predict(miami_test), columns = ['mine'], geometry = miami_test_chips['geometry'])
miami_test_probs = gpd.GeoDataFrame(az_clf.predict_proba(miami_test)[:,1], columns = ['mine_prob'], geometry=miami_test_chips['geometry'], crs = 'EPSG:4326')

miami_test_probs.to_file('miami_az_testing_heldout_1216')
miami_test_probs